In [ ]:
!nvidia-smi

Mon Jul 12 19:03:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install missingpy

     |████████████████████████████████| 51kB 5.6MB/s 


In [ ]:
pip install neural-tangents

     |████████████████████████████████| 122kB 13.0MB/s 


In [ ]:
# from data_generation import *
# from Matrix_Factorization import *
# from MICE import *
# from gain import *
# from misgan import *

import copy
import statistics
import numpy as np
import pandas as pd
import scipy
import time
import statsmodels.api as sm
from missingpy import *
from sklearn.linear_model import *
from sklearn.svm import *
from sklearn.ensemble import *
from sklearn.tree import *
from sklearn.neural_network import MLPRegressor
from numpy import linalg as LA
import pickle

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import grad
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import datasets, transforms

from matplotlib.patches import Rectangle
import pylab as plt
import torch.nn.functional as F
from sklearn.metrics import mean_squared_error
import math
from fractions import Fraction 

import jax.numpy
import neural_tangents as nt
from neural_tangents import stax
from jax import random
from jax.experimental import optimizers
# from jax.api import jit, grad, vmap
import functools

import warnings
warnings.filterwarnings("ignore")

n=200;p_f=p=250  #;miss_p=100
alpha=0.05

use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0' if use_cuda else 'cpu')
print(device)

cuda:0


In [ ]:
np.random.seed(1)
true_beta=np.zeros(p)
if p==250:
  q=[210,220,230] 
elif p==50:
  q=[40,44,48] 
elif p==500:
  q=[380,400,420]
elif p==1000:
  q=[650,700,750]    
elif p==2000:  
  q=[1500,1600,1700]     
elif p==1500:
  q=[1100,1200,1300]  
true_beta[q]=1
nonzero_true_beta=true_beta[true_beta!=0]

In [ ]:
def data_G_linear(n,p,true_beta,miss_pat1=[False,False,False,True,False],miss_pat2=[False,False,False,False,True],a0=0,a1=0.1,a2=0.1,a3=0,a4=0.1,a5=0.1,rho=0.1,seed=1,sigma1=0.2,sigma2=0.5):
  # obs is observed data matrix
  # mis is truth data matrix but masked
  # missing_row is indicator of whether a row has missing values
  # missing_col is indicator of whether a col has missing values
  # y is label
  # true_beta adds sparsity to model, necessary for inference
  np.random.seed(seed)
  data=np.zeros((n,p))
  data[:,0]=np.random.normal(size=(n,),scale=1)            #np.random.laplace   np.random.normal
  for col in range(1,p):
      data[:,col]=rho*data[:,col-1]+np.random.normal(size=(n,),scale=sigma1)

  # split to observed col and missing col
  missing_col1=np.array(miss_pat1*int(p/len(miss_pat1)))
  missing_col2=np.array(miss_pat2*int(p/len(miss_pat2)))
  obs=data[:,np.logical_or(missing_col1,missing_col2)==0]
  mis1=data[:,missing_col1==1]
  mis2=data[:,missing_col2==1]
  y=np.dot(np.concatenate((obs,mis1,mis2),axis=1),true_beta)+np.random.normal(size=(n,),scale=sigma2)
  ### MNAR 
  # logit1=a0+a1*np.mean(mis2,axis=1)+a2*y   # [0:5]
  # logit2=a3+a4*np.mean(mis1,axis=1)+a5*y
  ### MAR
  logit1=a0+a1*(np.mean(obs,axis=1))+a2*y
  logit2=a3+a4*(np.mean(obs,axis=1))+a5*y
  p_miss1 = 1/(1+np.exp(-logit1))  
  p_miss2 = 1/(1+np.exp(-logit2))

  missing_row1=np.zeros((n,))
  missing_row2=np.zeros((n,))
  for i in range(n):
    missing_row1[i]= np.random.choice(2, 1, p=[1-p_miss1[i],p_miss1[i]])
    missing_row2[i]= np.random.choice(2, 1, p=[1-p_miss2[i],p_miss2[i]])
  ### MCAR
  # p_miss1 = 0.7
  # p_miss2 = 0.6
  # for i in range(n):
  #   missing_row1[i]= np.random.choice(2, 1, p=[1-p_miss1,p_miss1])
  #   missing_row2[i]= np.random.choice(2, 1, p=[1-p_miss2,p_miss2])
  miss_col1=np.zeros(p+1)
  miss_col1[(p-sum(missing_col1)-sum(missing_col2)):(p-sum(missing_col2))]=1
  miss_col2=np.zeros(p+1)
  miss_col2[(p-sum(missing_col2)):-1]=1
 
  return obs,mis1,mis2,missing_row1==1,missing_row2==1,miss_col1==1,miss_col2==1,y.reshape((-1,1))

In [ ]:
def data_to_stats(imputed_data_list,true_beta,truth1,truth2,rows1,rows2,cols1,cols2):
    rounds=len(imputed_data_list)
    valid_p=sum(true_beta!=0)
    MSE=0
    MEANs=np.zeros((rounds,valid_p))
    within_VAR=np.zeros((valid_p,valid_p))
    Betw_VAR=np.zeros((valid_p,valid_p))

    for i in range(rounds):
      imputed_data_MICE=imputed_data_list[i]
      X=imputed_data_MICE[:,:-1]
      ols = sm.OLS(imputed_data_MICE[:,-1], X[:,true_beta!=0])
      result=ols.fit()
      within_VAR+=result.cov_params()/rounds
      MEANs[i,:]=result.params
      # print(imputed_data_MICE.shape)
      # print(imputed_data_MICE[:,cols1].shape)
      MSE+=np.sum((imputed_data_MICE[rows1,:][:,cols1]-truth1)**2)/(rounds*(np.sum(rows1)*np.sum(cols1)+np.sum(rows2)*np.sum(cols2)))
      MSE+=np.sum((imputed_data_MICE[rows2,:][:,cols2]-truth2)**2)/(rounds*(np.sum(rows1)*np.sum(cols1)+np.sum(rows2)*np.sum(cols2)))

    if rounds==1:
      VAR=within_VAR
      MEAN=MEANs[0,:]
    else:
      # Rubin's rule for total variance
      MEAN=np.mean(MEANs,axis=0)
      # between imputation variance
      for i in range(rounds):
          temp=MEANs[i,:]-MEAN
          Betw_VAR+=np.outer(temp,temp)/(rounds-1)
      VAR=within_VAR+(1+1/rounds)*Betw_VAR
    return MSE,MEAN,VAR,np.sqrt(VAR[0][0])

def Performance(MEAN,VAR,true_beta,alpha=0.05):
    nonzero_true_beta=true_beta[true_beta!=0]
    SE=np.sqrt(np.diagonal(VAR))
    # elementwise confidence interval
    conf=scipy.stats.norm.interval(1-alpha, loc=MEAN[0], scale=SE[0])
    coverage=((nonzero_true_beta[0]>conf[0])&(nonzero_true_beta[0]<conf[1]))
    bias=nonzero_true_beta[0]-MEAN[0]
    return bias,coverage

# ***Baseline***

In [ ]:
super=time.time()
full_SD=[]
partial_SD=[]
naive_imp_SD=[]

full_SE=0
partial_SE=0
naive_imp_SE=0

naive_imp_MSE=0

full_bias=0
partial_bias=0
naive_imp_bias=0

full_coverage=0
partial_coverage=0
naive_imp_coverage=0

num_of_missing_row = 0
num_of_missing_row1 = 0
num_of_missing_row2 = 0

a0=1
a1=-2
a2=3
a3=0
a4=2
a5=-2
rho=0.95
sigma1=0.1
sigma2=0.5

Monte_Carlo =500
for iteration in range(1,1+Monte_Carlo):
  obs,mis1,mis2,missing_row1,missing_row2,missing_col1,missing_col2,y=data_G_linear(n=n,p=p,true_beta=true_beta,seed=iteration,a0=a0,a1=a1,a2=a2,a3=a3,a4=a4,a5=a5,rho=rho,sigma1=sigma1,sigma2=sigma2)
  # print(sum(missing_row))
  truth1=mis1[missing_row1==True,:]
  truth2=mis2[missing_row2==True,:]

  full=np.concatenate((obs,mis1,mis2,y),axis=1)
  partial=full[np.logical_or(missing_row1,missing_row2)==0,:].copy()
  
  total_row = np.sum(np.logical_or(missing_row1, missing_row2))
  row1 = np.sum(missing_row1)
  row2 = np.sum(missing_row2)
  num_of_missing_row1 = (num_of_missing_row1*(iteration-1)+row1)/iteration
  num_of_missing_row2 = (num_of_missing_row2*(iteration-1)+row2)/iteration
  num_of_missing_row = (num_of_missing_row*(iteration-1)+total_row)/iteration
  ########## benchmark: full data
  s=time.time()
  mse,b,c,se=data_to_stats([full],true_beta,truth1,truth2,missing_row1,missing_row2,missing_col1,missing_col2)
  bias,cover=Performance(b,c,true_beta,alpha)
  
  full_SE=(full_SE*(iteration-1)+se)/iteration
  full_SD.append(b[0])
  full_bias=(full_bias*(iteration-1)+bias)/iteration
  full_coverage=(full_coverage*(iteration-1)+cover)/iteration

  ########## benchmark: partial
  s=time.time()
  X=partial[:,:-1]
  ols = sm.OLS(partial[:,-1], X[:,true_beta!=0])
  result=ols.fit()
  c=result.cov_params()
  b=result.params
  bias,cover=Performance(b,c,true_beta,alpha)
  
  partial_SE=(partial_SE*(iteration-1)+np.sqrt(c[0][0]))/iteration
  partial_SD.append(b[0])
  partial_bias=(partial_bias*(iteration-1)+bias)/iteration
  partial_coverage=(partial_coverage*(iteration-1)+cover)/iteration

  ########## benchmark: naive imputation
  mis1[missing_row1,:]=np.mean(mis1[missing_row1==0,:], axis=0)
  mis2[missing_row2,:]=np.mean(mis2[missing_row2==0,:], axis=0)
  naive_impute=np.concatenate((obs,mis1,mis2,y),axis=1)

  s=time.time()
  mse,b,c,se=data_to_stats([naive_impute],true_beta,truth1,truth2,missing_row1,missing_row2,missing_col1,missing_col2)
  bias,cover=Performance(b,c,true_beta,alpha)
  
  naive_imp_SE=(naive_imp_SE*(iteration-1)+se)/iteration
  naive_imp_SD.append(b[0])
  naive_imp_MSE=(naive_imp_MSE*(iteration-1)+mse)/iteration
  naive_imp_bias=(naive_imp_bias*(iteration-1)+bias)/iteration
  naive_imp_coverage=(naive_imp_coverage*(iteration-1)+cover)/iteration

print('full_MSE: NA ', 'full_bias ', full_bias, 'full_coverage ', full_coverage, 'full_SE: ',full_SE)
print('partial_MSE: NA ', 'partial_bias ', partial_bias, 'partial_coverage ', partial_coverage,'partial_SE: ',partial_SE)
print('naive_imp_MSE: ', naive_imp_MSE, 'naive_imp_bias ', naive_imp_bias, 'naive_imp_coverage ', naive_imp_coverage,'naive_imp_SE: ',naive_imp_SE)
full_StD=np.std(full_SD, dtype=np.float64)
partial_StD=np.std(partial_SD, dtype=np.float64)
naive_imp_StD=np.std(naive_imp_SD, dtype=np.float64)
print('full_StD: ', full_StD)
print('partial_StD: ', partial_StD)
print('naive_imp_StD: ', naive_imp_StD)

# missing rate
print('num_of_missing_row: ',num_of_missing_row)
print('num_of_missing_row1: ',num_of_missing_row1)
print('num_of_missing_row2: ',num_of_missing_row2)
print('n:', n, ' p: ',p, ' Monte Carlo:',Monte_Carlo)
print('a0=',a0,' a1=', a1,' a2=',a2,' a3=',a3,' a4=',a4,' a5=',a5,' rho=',rho,' sigma1=',sigma1,' sigma2=',sigma2)

full_MSE: NA  full_bias  0.005586164390235798 full_coverage  0.934 full_SE:  0.11029213518978898
partial_MSE: NA  partial_bias  0.23691621455169543 partial_coverage  0.8599999999999999 partial_SE:  0.33499818586440816
naive_imp_MSE:  0.1407233122435843 naive_imp_bias  0.32824648061145134 naive_imp_coverage  0.7279999999999995 naive_imp_SE:  0.220339447967689
full_StD:  0.11628026649469159
partial_StD:  0.3392245792851725
naive_imp_StD:  0.17650471074321245
num_of_missing_row:  179.79999999999998
num_of_missing_row1:  127.95200000000011
num_of_missing_row2:  99.74999999999999
n: 200  p:  250  Monte Carlo: 500
a0= 1  a1= -2  a2= 3  a3= 0  a4= 2  a5= -2  rho= 0.95  sigma1= 0.1  sigma2= 0.5


In [ ]:
#########    MAR     ###############
full_MSE: NA  full_bias  0.0025348257248207157 full_coverage  0.98 full_SE:  0.06051432012968249
partial_MSE: NA  partial_bias  0.18698975081304503 partial_coverage  0.79 partial_SE:  0.22982732749304483
naive_imp_MSE:  0.47165325803517083 naive_imp_bias  0.5312439428319942 naive_imp_coverage  0.2800000000000001 naive_imp_SE:  0.224208477812642
full_StD:  0.05248328969291819
partial_StD:  0.24195935511710312
naive_imp_StD:  0.17297712182094002
num_of_missing_row:  183.54000000000005
num_of_missing_row1:  119.38
num_of_missing_row2:  100.20999999999997
n: 200  p:  50  Monte Carlo: 100
a0= 1  a1= -2  a2= 3  a3= 0  a4= 2  a5= -2  rho= 0.95  sigma1= 0.1  sigma2= 0.5

full_MSE: NA  full_bias  -0.007575247672779701 full_coverage  0.94 full_SE:  0.11135577981887032
partial_MSE: NA  partial_bias  0.24429613699649244 partial_coverage  0.88 partial_SE:  0.3768127604475162
naive_imp_MSE:  0.1357637458435397 naive_imp_bias  0.0504011604760012 naive_imp_coverage  0.95 naive_imp_SE:  0.35734379739440675
full_StD:  0.11462733883297557
partial_StD:  0.39464189896035534
naive_imp_StD:  0.3203143087937309
num_of_missing_row:  183.51
num_of_missing_row1:  122.12
num_of_missing_row2:  100.99
n: 200  p:  500  Monte Carlo: 100
a0= 1  a1= -2  a2= 3  a3= 0  a4= 2  a5= -2  rho= 0.95  sigma1= 0.1  sigma2= 0.5

full_MSE: NA  full_bias  -0.010255731336393382 full_coverage  0.96 full_SE:  0.11124314688497172
partial_MSE: NA  partial_bias  0.3117022720433771 partial_coverage  0.89 partial_SE:  0.44217351404795807
naive_imp_MSE:  0.11453041579688575 naive_imp_bias  -0.005373166073109417 naive_imp_coverage  1.0 naive_imp_SE:  0.3515851722649606
full_StD:  0.11159661307137475
partial_StD:  0.46222097716954874
naive_imp_StD:  0.28244059935672416
num_of_missing_row:  184.07000000000005
num_of_missing_row1:  122.53
num_of_missing_row2:  99.79
n: 200  p:  1500  Monte Carlo: 100
a0= 1  a1= -2  a2= 3  a3= 0  a4= 2  a5= -2  rho= 0.95  sigma1= 0.1  sigma2= 0.5

full_MSE: NA  full_bias  -0.00848520442899759 full_coverage  0.94 full_SE:  0.11063839584248202
partial_MSE: NA  partial_bias  0.2896368381691573 partial_coverage  0.85 partial_SE:  0.3867348992395619
naive_imp_MSE:  0.11167624768442445 naive_imp_bias  0.048357738044061765 naive_imp_coverage  0.99 naive_imp_SE:  0.3490580771435289
full_StD:  0.10835304405249002
partial_StD:  0.4528479315825961
naive_imp_StD:  0.2928267597923345
num_of_missing_row:  183.72999999999996
num_of_missing_row1:  121.75
num_of_missing_row2:  100.13
n: 200  p:  2000  Monte Carlo: 100
a0= 1  a1= -2  a2= 3  a3= 0  a4= 2  a5= -2  rho= 0.95  sigma1= 0.1  sigma2= 0.5

In [ ]:
#########    MCAR     ###############
full_MSE: NA  full_bias  0.03501496966150251 full_coverage  0.94 full_SE:  0.1122019739713618
partial_MSE: NA  partial_bias  0.05181274769596977 partial_coverage  0.94 partial_SE:  0.3513214580049656
naive_imp_MSE:  0.11219441792543149 naive_imp_bias  0.0631547394095697 naive_imp_coverage  0.99 naive_imp_SE:  0.2770932725046795
full_StD:  0.11730948359519462
partial_StD:  0.4021832251760242
naive_imp_StD:  0.21009377262087947
num_of_missing_row:  174.78
num_of_missing_row1:  138.12999999999988
num_of_missing_row2:  119.29
n: 200  p:  1000  Monte Carlo: 100
a0= 1  a1= -2  a2= 2  a3= 0  a4= 2  a5= -2  rho= 0.95  sigma1= 0.1  sigma2= 0.5

In [ ]:
full_MSE: NA  full_bias  0.03501496966150251 full_coverage  0.94 full_SE:  0.11220197397136182
partial_MSE: NA  partial_bias  0.20442581162484252 partial_coverage  0.86 partial_SE:  0.4037255022680312
naive_imp_MSE:  0.11694542407684264 naive_imp_bias  0.5131150279866664 naive_imp_coverage  0.51 naive_imp_SE:  0.2624244620052552
full_StD:  0.11730948359519462
partial_StD:  0.380017129958213
naive_imp_StD:  0.21494979685921692
num_of_missing_row:  182.05
num_of_missing_row1:  130.45000000000005
num_of_missing_row2:  100.26
n: 200  p:  1000  Monte Carlo: 100
a0= 1  a1= -2  a2= 2  a3= 0  a4= 2  a5= -2  rho= 0.95  sigma1= 0.1  sigma2= 0.5
#####                   MNAR               ###############
full_MSE: NA  full_bias  0.0025348257248207157 full_coverage  0.98 full_SE:  0.06051432012968249
partial_MSE: NA  partial_bias  0.21435602516308996 partial_coverage  0.78 partial_SE:  0.23407863496403422
naive_imp_MSE:  0.4772731772601524 naive_imp_bias  0.5597627357786974 naive_imp_coverage  0.24000000000000005 naive_imp_SE:  0.2246416126534651
full_StD:  0.05248328969291819
partial_StD:  0.2401891991411309
naive_imp_StD:  0.17203557970217498
num_of_missing_row:  183.75000000000003
num_of_missing_row1:  119.02
num_of_missing_row2:  100.28
n: 200  p:  50  Monte Carlo: 100
a0= 1  a1= -2  a2= 3  a3= 0  a4= 2  a5= -2  rho= 0.95  sigma1= 0.1  sigma2= 0.5

full_MSE: NA  full_bias  -0.002721816608919814 full_coverage  0.9000000000000001 full_SE:  0.10985858010909645
partial_MSE: NA  partial_bias  0.25185782377632454 partial_coverage  0.89 partial_SE:  0.33857990870083077
naive_imp_MSE:  0.14142110117599305 naive_imp_bias  0.35395471954971763 naive_imp_coverage  0.7199999999999999 naive_imp_SE:  0.2210570628568485
full_StD:  0.11413164193102622
partial_StD:  0.3319507401756726
naive_imp_StD:  0.17127344412593687
num_of_missing_row:  180.27000000000004
num_of_missing_row1:  127.76999999999998
num_of_missing_row2:  100.01
n: 200  p:  250  Monte Carlo: 100
a0= 1  a1= -2  a2= 3  a3= 0  a4= 2  a5= -2  rho= 0.95  sigma1= 0.1  sigma2= 0.5

full_MSE: NA  full_bias  0.03501496966150251 full_coverage  0.94 full_SE:  0.11220197397136182
partial_MSE: NA  partial_bias  0.2824804830031145 partial_coverage  0.7599999999999999 partial_SE:  0.34479927429380547
naive_imp_MSE:  0.11309560977924807 naive_imp_bias  0.7022317573458021 naive_imp_coverage  0.11 naive_imp_SE:  0.25722431667802853
full_StD:  0.11730948359519462
partial_StD:  0.4201487388582143
naive_imp_StD:  0.1941220508957793
num_of_missing_row:  179.45
num_of_missing_row1:  127.02
num_of_missing_row2:  99.86
n: 200  p:  1000  Monte Carlo: 100
a0= 1  a1= -2  a2= 3  a3= 0  a4= 2  a5= -2  rho= 0.95  sigma1= 0.1  sigma2= 0.5

# ***GAIN***

In [ ]:
from gain import gain
from utils import rmse_loss

Model_2_SD=[]
Model_2_SE_list=[]
Model_2_SE=0
Model_2_MSE=0
Model_2_bias=0
Model_2_coverage=0
Model_2_time=0

a0=1
a1=-2
a2=3
a3=0
a4=2
a5=-2
rho=0.95
sigma1=0.1
sigma2=0.5
Monte_Carlo =100

for iteration in range(1,1+Monte_Carlo):
  obs,mis1,mis2,missing_row1,missing_row2,missing_col1,missing_col2,y=data_G_linear(n=n,p=p,true_beta=true_beta,seed=iteration,a0=a0,a1=a1,a2=a2,a3=a3,a4=a4,a5=a5,rho=rho,sigma1=sigma1,sigma2=sigma2)
  truth1=mis1[missing_row1==True,:].copy()
  truth2=mis2[missing_row2==True,:].copy()
  
  s=time.time()

  mis1[missing_row1==True,:]=np.nan
  mis2[missing_row2==True,:]=np.nan
  rawdata = np.concatenate((obs,mis1,mis2,y),axis=1).copy()
  
  gain_parameters = {'batch_size': 64,
                  'hint_rate': 0.9,
                  'alpha': 1.0,
                  'iterations': 10000}
  imputed_data_x = gain(rawdata.copy(), gain_parameters)

  mse,b,c,se=data_to_stats([imputed_data_x],true_beta,truth1,truth2,missing_row1,missing_row2,missing_col1,missing_col2)
  bias,cover=Performance(b,c,true_beta,alpha)
  print('Model_2 cost ',time.time()-s,' sec \n','MSE:', mse,'; Bias: ',bias,'; Coverage: ',cover)
  Model_2_SE_list.append(se)
  Model_2_SE=(Model_2_SE*(iteration-1)+se)/iteration
  Model_2_SD.append(b[0])
  Model_2_MSE=(Model_2_MSE*(iteration-1)+mse)/iteration
  Model_2_bias=(Model_2_bias*(iteration-1)+bias)/iteration
  Model_2_coverage=(Model_2_coverage*(iteration-1)+cover)/iteration
  Model_2_time = (Model_2_time*(iteration-1)+time.time()-s)/iteration

  if iteration%5==0:
    print('iteration: ',iteration, 'Model_2_MSE: ',Model_2_MSE,'Model_2_bias: ',Model_2_bias,'Model_2_coverage: ',Model_2_coverage,'Model_2_SE: ',Model_2_SE)
    
    Model_2_StD=np.std(Model_2_SD, dtype=np.float64)
    print('Model_2_StD: ',Model_2_StD)
    Model_2_SE_median=statistics.median(Model_2_SE_list)
    print('Model_2 average cost:', Model_2_time)

    print('n:', n, ' p: ',p, ' Monte Carlo:',Monte_Carlo)
    print('a0=',a0,' a1=', a1,' a2=',a2,' a3=',a3,' a4=',a4,' a5=',a5,' rho=',rho,' sigma1=',sigma1,' sigma2=',sigma2)

Instructions for updating:
non-resource variables are not supported in the long term


100%|██████████| 10000/10000 [01:38<00:00, 101.71it/s]


Model_2 cost  99.8533787727356  sec 
 MSE: 0.75687349301031 ; Bias:  1.6324934279263068 ; Coverage:  False


100%|██████████| 10000/10000 [01:37<00:00, 102.24it/s]


Model_2 cost  98.85503458976746  sec 
 MSE: 0.7564085098152966 ; Bias:  1.466665029815354 ; Coverage:  False


...

Model_2 cost  103.24056506156921  sec 
 MSE: 0.7807894673182486 ; Bias:  0.8958931204405839 ; Coverage:  False


100%|██████████| 10000/10000 [01:38<00:00, 101.79it/s]


Model_2 cost  104.01379919052124  sec 
 MSE: 0.7683458834887136 ; Bias:  0.2534075716649413 ; Coverage:  False


 44%|████▎     | 4352/10000 [00:42<00:54, 102.99it/s]

KeyboardInterrupt: ignored

In [ ]:
##### MAR #####
iteration:  100 Model_2_MSE:  1.3560164769618632 Model_2_bias:  0.3213163922483485 Model_2_coverage:  0.38000000000000006 Model_2_SE:  0.11425713536740449
Model_2_StD:  0.42629791528092675
Model_2 average cost: 34.625168442726135
n: 200  p:  50  Monte Carlo: 100
a0= 1  a1= -2  a2= 3  a3= 0  a4= 2  a5= -2  rho= 0.95  sigma1= 0.1  sigma2= 0.5

iteration:  100 Model_2_MSE:  0.868519623542013 Model_2_bias:  0.6257938960994572 Model_2_coverage:  0.18000000000000005 Model_2_SE:  0.14630196611986063
Model_2_StD:  0.5424637137478175
Model_2 average cost: 39.06981110811233
n: 200  p:  250  Monte Carlo: 100
a0= 1  a1= -2  a2= 3  a3= 0  a4= 2  a5= -2  rho= 0.95  sigma1= 0.1  sigma2= 0.5

iteration:  65 Model_2_MSE:  0.7906958188677685 Model_2_bias:  0.6976235556145928 Model_2_coverage:  0.2461538461538462 Model_2_SE:  0.10989737736966337
Model_2_StD:  0.7274650834795663
Model_2 average cost: 48.58079683597271
n: 200  p:  500  Monte Carlo: 100

iteration:  25 Model_2_MSE:  0.7628725758647521 Model_2_bias:  0.9067473347554801 Model_2_coverage:  0.28 Model_2_SE:  0.09400274449945932
Model_2_StD:  0.7205141373425118
Model_2 average cost: 114.71695168495178
n: 200  p:  1500  Monte Carlo: 100
a0= 1  a1= -2  a2= 3  a3= 0  a4= 2  a5= -2  rho= 0.95  sigma1= 0.1  sigma2= 0.5

iteration:  100 Model_2_MSE:  0.7383403064925876 Model_2_bias:  0.6897764227348189 Model_2_coverage:  0.18000000000000005 Model_2_SE:  0.1697709171016964
Model_2_StD:  0.5693423386150692
Model_2 average cost: 98.04742572784424
n: 200  p:  1000  Monte Carlo: 100
a0= 1  a1= -2  a2= 3  a3= 0  a4= 2  a5= -2  rho= 0.95  sigma1= 0.1  sigma2= 0.5

In [ ]:
iteration:  100 Model_2_MSE:  0.7190594273913655 Model_2_bias:  0.5685083773487243 Model_2_coverage:  0.30000000000000004 Model_2_SE:  0.19027379712303272
Model_2_StD:  0.6155049856801302
Model_2 average cost: 101.22782071352005
n: 200  p:  1000  Monte Carlo: 100
a0= 1  a1= -2  a2= 2  a3= 0  a4= 2  a5= -2  rho= 0.95  sigma1= 0.1  sigma2= 0.5

##### MNAR #####
iteration:  100 Model_2_MSE:  1.4822396753689475 Model_2_bias:  0.44481586588300887 Model_2_coverage:  0.24000000000000007 Model_2_SE:  0.11876120953844352
Model_2_StD:  0.46417549181255285
Model_2 average cost: 35.9048989367485
n: 200  p:  50  Monte Carlo: 100
a0= 1  a1= -2  a2= 3  a3= 0  a4= 2  a5= -2  rho= 0.95  sigma1= 0.1  sigma2= 0.5

iteration:  100 Model_2_MSE:  0.8618507166789123 Model_2_bias:  0.6212997615311452 Model_2_coverage:  0.18 Model_2_SE:  0.1502809565022466
Model_2_StD:  0.5088585380570896
Model_2 average cost: 53.24450348854065
n: 200  p:  250  Monte Carlo: 100
a0= 1  a1= -2  a2= 3  a3= 0  a4= 2  a5= -2  rho= 0.95  sigma1= 0.1  sigma2= 0.5

iteration:  100 Model_2_MSE:  0.7395177331253409 Model_2_bias:  0.6488564603209248 Model_2_coverage:  0.18 Model_2_SE:  0.17198748060019273
Model_2_StD:  0.5830330156383707
Model_2 average cost: 111.13734125375748
n: 200  p:  1000  Monte Carlo: 100
a0= 1  a1= -2  a2= 3  a3= 0  a4= 2  a5= -2  rho= 0.95  sigma1= 0.1  sigma2= 0.5

# ***MICE***

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import *

def MICE(missing_data,estimator,seed=1,whether_MI=False):
    MICE_imputer=IterativeImputer(estimator=estimator,skip_complete=True,max_iter=5, tol=0.01,sample_posterior=whether_MI,random_state=seed)
    imputed_data_MICE=MICE_imputer.fit_transform(missing_data.copy())
    
    return imputed_data_MICE

In [ ]:
Model_2_SD=[]
Model_2_SE_list=[]
Model_2_SE=0
Model_2_MSE=0
Model_2_bias=0
Model_2_coverage=0
Model_2_time=0

a0=1
a1=-2
a2=3
a3=0
a4=2
a5=-2
rho=0.95
sigma1=0.1
sigma2=0.5
Monte_Carlo =100
MI =3

for iteration in range(1,1+Monte_Carlo):
  obs,mis1,mis2,missing_row1,missing_row2,missing_col1,missing_col2,y=data_G_linear(n=n,p=p,true_beta=true_beta,seed=iteration,a0=a0,a1=a1,a2=a2,a3=a3,a4=a4,a5=a5,rho=rho,sigma1=sigma1,sigma2=sigma2)
  truth1=mis1[missing_row1==True,:].copy()
  truth2=mis2[missing_row2==True,:].copy()
  
  s=time.time()

  mis1[missing_row1==True,:]=np.nan
  mis2[missing_row2==True,:]=np.nan
  rawdata = np.concatenate((obs,mis1,mis2,y),axis=1).copy()
  
  alist=[]
  for i in range(MI):
    data=MICE(rawdata.copy(),BayesianRidge(),seed=i,whether_MI=True)       
    alist.append(data)

  mse,b,c,se=data_to_stats(alist,true_beta,truth1,truth2,missing_row1,missing_row2,missing_col1,missing_col2)
  bias,cover=Performance(b,c,true_beta,alpha)
  print('Model_2 cost ',time.time()-s,' sec \n','MSE:', mse,'; Bias: ',bias,'; Coverage: ',cover)
  Model_2_SE_list.append(se)
  Model_2_SE=(Model_2_SE*(iteration-1)+se)/iteration
  Model_2_SD.append(b[0])
  Model_2_MSE=(Model_2_MSE*(iteration-1)+mse)/iteration
  Model_2_bias=(Model_2_bias*(iteration-1)+bias)/iteration
  Model_2_coverage=(Model_2_coverage*(iteration-1)+cover)/iteration
  Model_2_time = (Model_2_time*(iteration-1)+time.time()-s)/iteration

  if iteration%5==0:
    print('iteration: ',iteration, 'Model_2_MSE: ',Model_2_MSE,'Model_2_bias: ',Model_2_bias,'Model_2_coverage: ',Model_2_coverage,'Model_2_SE: ',Model_2_SE)
    
    Model_2_StD=np.std(Model_2_SD, dtype=np.float64)
    print('Model_2_StD: ',Model_2_StD)
    # Model_2_SE_median=statistics.median(Model_2_SE_list)
    print('Model_2 average cost:', Model_2_time/MI)

    print('n:', n, ' p: ',p, ' Monte Carlo:',Monte_Carlo,'MI:',MI)
    print('a0=',a0,' a1=', a1,' a2=',a2,' a3=',a3,' a4=',a4,' a5=',a5,' rho=',rho,' sigma1=',sigma1,' sigma2=',sigma2)

Model_2 cost  97.95703220367432  sec 
 MSE: 0.026394539269082237 ; Bias:  0.09128316607199938 ; Coverage:  True
Model_2 cost  95.49221706390381  sec 
 MSE: 0.0260120778959902 ; Bias:  0.10381988715311674 ; Coverage:  True
Model_2 cost  91.7993655204773  sec 
 MSE: 0.025409298758667657 ; Bias:  0.0853480257624768 ; Coverage:  True
Model_2 cost  93.03799295425415  sec 
 MSE: 0.026022775835384496 ; Bias:  0.020560371348346762 ; Coverage:  True
Model_2 cost  99.16528058052063  sec 
 MSE: 0.02706865613050196 ; Bias:  -0.049075044023000336 ; Coverage:  True
iteration:  5 Model_2_MSE:  0.02618146957792531 Model_2_bias:  0.05038728126258787 Model_2_coverage:  1.0 Model_2_SE:  0.10724037887729307
Model_2_StD:  0.05750159846270739
Model_2 average cost: 31.83052398363749
n: 200  p:  500  Monte Carlo: 100 MI: 3
a0= 1  a1= -2  a2= 3  a3= 0  a4= 2  a5= -2  rho= 0.95  sigma1= 0.1  sigma2= 0.5
Model_2 cost  80.5021424293518  sec 
 MSE: 0.027497918819246996 ; Bias:  -0.06066572137967596 ; Coverage:  Tr

In [ ]:
#################################### MAR #################################
iteration:  100 Model_2_MSE:  0.020005109410976482 Model_2_bias:  0.0031974062968884325 Model_2_coverage:  0.97 Model_2_SE:  0.06449844945915807
Model_2_StD:  0.05675406648193172
Model_2 average cost: 6.427422561645508
n: 200  p:  50  Monte Carlo: 100 MI: 3
a0= 1  a1= -2  a2= 3  a3= 0  a4= 2  a5= -2  rho= 0.95  sigma1= 0.1  sigma2= 0.5

iteration:  100 Model_2_MSE:  0.023410699138156348 Model_2_bias:  -0.0061003231592991645 Model_2_coverage:  0.93 Model_2_SE:  0.11672326714181189
Model_2_StD:  0.12130368359144336
Model_2 average cost: 111.16540475130081
n: 200  p:  250  Monte Carlo: 100 MI: 3
a0= 1  a1= -2  a2= 3  a3= 0  a4= 2  a5= -2  rho= 0.95  sigma1= 0.1  sigma2= 0.5

iteration:  100 Model_2_MSE:  0.02623461059918931 Model_2_bias:  -0.03209829852652116 Model_2_coverage:  0.9 Model_2_SE:  0.1180274566412788
Model_2_StD:  0.14189359430180165
Model_2 average cost: 32.78832807620366
n: 200  p:  500  Monte Carlo: 100 MI: 3
a0= 1  a1= -2  a2= 3  a3= 0  a4= 2  a5= -2  rho= 0.95  sigma1= 0.1  sigma2= 0.5

iteration:  100 Model_2_MSE:  0.03595457477262314 Model_2_bias:  0.010856693778450286 Model_2_coverage:  0.8200000000000002 Model_2_SE:  0.13463875470413975
Model_2_StD:  0.2047276547957849
Model_2 average cost: 125.88171967029571
n: 200  p:  1000  Monte Carlo: 100 MI: 1
a0= 1  a1= -2  a2= 3  a3= 0  a4= 2  a5= -2  rho= 0.95  sigma1= 0.1  sigma2= 0.5

In [ ]:
#################################### MNAR #################################
iteration:  100 Model_2_MSE:  0.023393896552039967 Model_2_bias:  -0.004964117330061298 Model_2_coverage:  0.93 Model_2_SE:  0.11603503893377896
Model_2_StD:  0.12447567814492734
Model_2 average cost: 486.21003947734835
n: 200  p:  250  Monte Carlo: 100
a0= 1  a1= -2  a2= 3  a3= 0  a4= 2  a5= -2  rho= 0.95  sigma1= 0.1  sigma2= 0.5

iteration:  100 Model_2_MSE:  0.01988731538371139 Model_2_bias:  0.0036740525194513345 Model_2_coverage:  0.98 Model_2_SE:  0.06364716635011386
Model_2_StD:  0.05599138862474952
Model_2 average cost: 21.363479385375978
n: 200  p:  50  Monte Carlo: 100
a0= 1  a1= -2  a2= 3  a3= 0  a4= 2  a5= -2  rho= 0.95  sigma1= 0.1  sigma2= 0.5


# ***softimpute***

In [ ]:
from softimpute import *
from utils import *

Model_2_SD=[]
Model_2_SE_list=[]
Model_2_SE=0
Model_2_MSE=0
Model_2_bias=0
Model_2_coverage=0
Model_2_time=0

a0=1
a1=-2
a2=3
a3=0
a4=2
a5=-2
rho=0.95
sigma1=0.1
sigma2=0.5
Monte_Carlo =100

grid_len =5
maxit = 500
thresh = 1e-5

for iteration in range(1,1+Monte_Carlo):
  obs,mis1,mis2,missing_row1,missing_row2,missing_col1,missing_col2,y=data_G_linear(n=n,p=p,true_beta=true_beta,seed=iteration,a0=a0,a1=a1,a2=a2,a3=a3,a4=a4,a5=a5,rho=rho,sigma1=sigma1,sigma2=sigma2)
  truth1=mis1[missing_row1==True,:].copy()
  truth2=mis2[missing_row2==True,:].copy()

  mis1[missing_row1==True,:]=np.nan
  mis2[missing_row2==True,:]=np.nan
  rawdata = np.concatenate((obs,mis1,mis2,y),axis=1).copy()
  
  cv_error, grid_lambda = cv_softimpute(rawdata.copy(), grid_len = grid_len, maxit = 50, thresh=1e-3)
  s=time.time()
  U_thresh, imp = softimpute(rawdata.copy(), grid_lambda[np.argmin(cv_error)], maxit = maxit, thresh=thresh)
  alist=[imp]

  mse,b,c,se=data_to_stats(alist,true_beta,truth1,truth2,missing_row1,missing_row2,missing_col1,missing_col2)
  bias,cover=Performance(b,c,true_beta,alpha)
  print('Model_2 cost ',time.time()-s,' sec \n','MSE:', mse,'; Bias: ',bias,'; Coverage: ',cover)
  Model_2_SE_list.append(se)
  Model_2_SE=(Model_2_SE*(iteration-1)+se)/iteration
  Model_2_SD.append(b[0])
  Model_2_MSE=(Model_2_MSE*(iteration-1)+mse)/iteration
  Model_2_bias=(Model_2_bias*(iteration-1)+bias)/iteration
  Model_2_coverage=(Model_2_coverage*(iteration-1)+cover)/iteration
  Model_2_time = (Model_2_time*(iteration-1)+time.time()-s)/iteration

  if iteration%5==0:
    print('iteration: ',iteration, 'Model_2_MSE: ',Model_2_MSE,'Model_2_bias: ',Model_2_bias,'Model_2_coverage: ',Model_2_coverage,'Model_2_SE: ',Model_2_SE)
    
    Model_2_StD=np.std(Model_2_SD, dtype=np.float64)
    print('Model_2_StD: ',Model_2_StD)
    # Model_2_SE_median=statistics.median(Model_2_SE_list)
    print('Model_2 average cost:', Model_2_time)

    print('n:', n, ' p: ',p, ' Monte Carlo:',Monte_Carlo, 'grid_len:',grid_len,'maxit:',maxit,'thresh:',thresh)
    print('a0=',a0,' a1=', a1,' a2=',a2,' a3=',a3,' a4=',a4,' a5=',a5,' rho=',rho,' sigma1=',sigma1,' sigma2=',sigma2)

# ***optimal transport-sinkhorn***

In [ ]:
pip install geomloss

In [ ]:
from imputers import *
from utils import *
torch.set_default_tensor_type('torch.DoubleTensor')

Model_2_SD=[]
Model_2_SE_list=[]
Model_2_SE=0
Model_2_MSE=0
Model_2_bias=0
Model_2_coverage=0
Model_2_time=0

a0=1
a1=-2
a2=3
a3=0
a4=2
a5=-2
rho=0.95
sigma1=0.1
sigma2=0.5
MI = 3
Monte_Carlo =50

for iteration in range(1,1+Monte_Carlo):
  obs,mis1,mis2,missing_row1,missing_row2,missing_col1,missing_col2,y=data_G_linear(n=n,p=p,true_beta=true_beta,seed=iteration,a0=a0,a1=a1,a2=a2,a3=a3,a4=a4,a5=a5,rho=rho,sigma1=sigma1,sigma2=sigma2)
  truth1=mis1[missing_row1==True,:].copy()
  truth2=mis2[missing_row2==True,:].copy()
  
  s=time.time()

  mis1[missing_row1==True,:]=np.nan
  mis2[missing_row2==True,:]=np.nan
  rawdata = np.concatenate((obs,mis1,mis2,y),axis=1).copy()
  
  rawdata = torch.from_numpy(rawdata)
  epsilon = pick_epsilon(rawdata)
  alist=[]
  sk_imputer = OTimputer(eps=epsilon)
  for i in range(MI):
    sk_imp = sk_imputer.fit_transform(rawdata)
    data = sk_imp.detach().cpu().numpy()     
    alist.append(data)

  mse,b,c,se=data_to_stats(alist,true_beta,truth1,truth2,missing_row1,missing_row2,missing_col1,missing_col2)
  bias,cover=Performance(b,c,true_beta,alpha)
  print('Model_2 cost ',time.time()-s,' sec \n','MSE:', mse,'; Bias: ',bias,'; Coverage: ',cover)
  Model_2_SE_list.append(se)
  Model_2_SE=(Model_2_SE*(iteration-1)+se)/iteration
  Model_2_SD.append(b[0])
  Model_2_MSE=(Model_2_MSE*(iteration-1)+mse)/iteration
  Model_2_bias=(Model_2_bias*(iteration-1)+bias)/iteration
  Model_2_coverage=(Model_2_coverage*(iteration-1)+cover)/iteration
  Model_2_time = (Model_2_time*(iteration-1)+time.time()-s)/iteration

  if iteration%5==0:
    print('iteration: ',iteration, 'Model_2_MSE: ',Model_2_MSE,'Model_2_bias: ',Model_2_bias,'Model_2_coverage: ',Model_2_coverage,'Model_2_SE: ',Model_2_SE)
    
    Model_2_StD=np.std(Model_2_SD, dtype=np.float64)
    print('Model_2_StD: ',Model_2_StD)
    # Model_2_SE_median=statistics.median(Model_2_SE_list)
    print('Model_2 average cost:', Model_2_time)

    print('n:', n, ' p: ',p, ' Monte Carlo:',Monte_Carlo, 'MI',MI)
    print('a0=',a0,' a1=', a1,' a2=',a2,' a3=',a3,' a4=',a4,' a5=',a5,' rho=',rho,' sigma1=',sigma1,' sigma2=',sigma2)

# ***optimal transport-round rubin***

In [ ]:
pip install geomloss

In [ ]:
from imputers import *
from utils import *
torch.set_default_tensor_type('torch.DoubleTensor')

Model_2_SD=[]
Model_2_SE_list=[]
Model_2_SE=0
Model_2_MSE=0
Model_2_bias=0
Model_2_coverage=0
Model_2_time=0

a0=1
a1=-2
a2=3
a3=0
a4=2
a5=-2
rho=0.95
sigma1=0.1
sigma2=0.5
Monte_Carlo =20

for iteration in range(1,1+Monte_Carlo):
  obs,mis1,mis2,missing_row1,missing_row2,missing_col1,missing_col2,y=data_G_linear(n=n,p=p,true_beta=true_beta,seed=iteration,a0=a0,a1=a1,a2=a2,a3=a3,a4=a4,a5=a5,rho=rho,sigma1=sigma1,sigma2=sigma2)
  truth1=mis1[missing_row1==True,:].copy()
  truth2=mis2[missing_row2==True,:].copy()
  
  s=time.time()

  mis1[missing_row1==True,:]=np.nan
  mis2[missing_row2==True,:]=np.nan
  rawdata = np.concatenate((obs,mis1,mis2,y),axis=1).copy()
  rawdata = torch.from_numpy(rawdata)

  alist=[]
  models = {}
  for i in range(p+1):
    models[i] = nn.Linear(p, 1)
  epsilon = pick_epsilon(rawdata)

  for i in range(1):
    lin_rr_imputer = RRimputer(models, eps=epsilon)
    lin_rr_imp = lin_rr_imputer.fit_transform(rawdata)
    data = lin_rr_imp.detach().numpy()     
    alist.append(data)

  mse,b,c,se=data_to_stats(alist,true_beta,truth1,truth2,missing_row1,missing_row2,missing_col1,missing_col2)
  bias,cover=Performance(b,c,true_beta,alpha)
  print('Model_2 cost ',time.time()-s,' sec \n','MSE:', mse,'; Bias: ',bias,'; Coverage: ',cover)
  Model_2_SE_list.append(se)
  Model_2_SE=(Model_2_SE*(iteration-1)+se)/iteration
  Model_2_SD.append(b[0])
  Model_2_MSE=(Model_2_MSE*(iteration-1)+mse)/iteration
  Model_2_bias=(Model_2_bias*(iteration-1)+bias)/iteration
  Model_2_coverage=(Model_2_coverage*(iteration-1)+cover)/iteration
  Model_2_time = (Model_2_time*(iteration-1)+time.time()-s)/iteration

  if iteration%5==0:
    print('iteration: ',iteration, 'Model_2_MSE: ',Model_2_MSE,'Model_2_bias: ',Model_2_bias,'Model_2_coverage: ',Model_2_coverage,'Model_2_SE: ',Model_2_SE)
    
    Model_2_StD=np.std(Model_2_SD, dtype=np.float64)
    print('Model_2_StD: ',Model_2_StD)
    Model_2_SE_median=statistics.median(Model_2_SE_list)
    print('Model_2 average cost:', Model_2_time)

print('n:', n, ' p: ',p, ' Monte Carlo:',Monte_Carlo)
print('a0=',a0,' a1=', a1,' a2=',a2,' a3=',a3,' a4=',a4,' a5=',a5,' rho=',rho,' sigma1=',sigma1,' sigma2=',sigma2)

# ***MIGAN2***

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import grad
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data import WeightedRandomSampler
from torchvision import datasets, transforms
import torch.nn.functional as F

In [ ]:
class Generate_data_model(Dataset):   
  def __init__(self, array, transform=None):  
    self.array = array
    self.transform = transform

  def __len__(self):
    return len(self.array)

  def __getitem__(self, index):
    return self.array[index]

class Generate_data_model_2(Dataset):   
  def __init__(self, array, mask, transform=None):  
    self.array = array
    self.mask = mask             
    self.transform = transform

  def __len__(self):
    return len(self.array)

  def __getitem__(self, index):
    return self.array[index], self.mask

class Imputer_model_2(nn.Module):
  def __init__(self, n1=251, n2=200, n3=100, n4=100):
    super().__init__()
    self.fc = nn.Sequential(
        nn.Linear(n1, n2),
        # nn.ReLU(),
        nn.Tanh(),
        nn.Linear(n2, n3),
        # nn.ReLU(),
        nn.Tanh(),
        nn.Linear(n3, n4),
    )
  def forward(self, data, mask, noise):
    net = data * (1 - mask) + noise * mask
    net = net.view(data.shape[0], -1)
    net = self.fc(net.float())
    net = net.view(data.shape[0], -1)
    return net

class Critic_model_2(nn.Module):
  def __init__(self, n1=251, n2=200, n3=50, n4=1):  #n1=251, n2=100, n3=20, n4=1
    super().__init__()
    self.fc = nn.Sequential(
        nn.Linear(n1, n2),
        nn.ReLU(),   
        # nn.Tanh(),
        nn.Linear(n2, n3),
        nn.ReLU(),   
        # nn.Tanh(),
        nn.Linear(n3, n4),
    )
  def forward(self, x):
    out = self.fc(x)
    return out


class CriticUpdater_model:
  def __init__(self, critic, critic_optimizer, batch_size=8, gp_lambda=10, device='cuda:0'):
    self.critic = critic
    self.critic_optimizer = critic_optimizer
    self.gp_lambda = gp_lambda
    # Interpolation coefficient
    self.eps = torch.empty(batch_size, 1,device=device)
    # For computing the gradient penalty
    self.ones = torch.ones(batch_size, 1).to(device)

  def __call__(self, real, fake, device='cuda:0'):
    real = real.detach()
    fake = fake.detach()
    self.critic.zero_grad()
    self.eps.uniform_(0, 1)
    interp = (self.eps * real + (1 - self.eps) * fake).to(device).requires_grad_()
    grad_d = grad(outputs=self.critic(interp.float()), inputs=interp, grad_outputs=self.ones,create_graph=True)[0]  #grad_outputs=self.ones,create_graph=True
    grad_d = grad_d.view(real.shape[0], -1)
    grad_penalty = ((grad_d.norm(dim=1) - 1)**2).mean() * self.gp_lambda
    w_dist = self.critic(fake.float()).mean() - self.critic(real.float()).mean()
    loss = w_dist + grad_penalty
    loss.backward()
    self.critic_optimizer.step()

class CriticUpdater_model_2:
  def __init__(self, critic, critic_optimizer, batch_size=8, gp_lambda=10, device='cuda:0'):
    self.critic = critic
    self.critic_optimizer = critic_optimizer
    self.gp_lambda = gp_lambda
    # Interpolation coefficient
    self.eps = torch.empty(batch_size, 1,device=device)
    # For computing the gradient penalty
    self.ones = torch.ones(batch_size, 1).to(device)

  def __call__(self, real, fake1, fake2, fake3, device='cuda:0'):
    real = real.detach()
    fake1 = fake1.detach()
    fake2 = fake2.detach()
    fake3 = fake3.detach()
    self.critic.zero_grad()
    self.eps.uniform_(0, 1)
    interp1 = (self.eps * real + (1 - self.eps) * fake1).to(device).requires_grad_()
    self.eps.uniform_(0, 1)
    interp2 = (self.eps * real + (1 - self.eps) * fake2).to(device).requires_grad_()
    self.eps.uniform_(0, 1)
    interp3 = (self.eps * real + (1 - self.eps) * fake3).to(device).requires_grad_()
    grad_d1 = grad(outputs=self.critic(interp1.float()), inputs=interp1, grad_outputs=self.ones,create_graph=True)[0].view(real.shape[0], -1)  #grad_outputs=self.ones,create_graph=True
    grad_d2 = grad(outputs=self.critic(interp2.float()), inputs=interp2, grad_outputs=self.ones,create_graph=True)[0].view(real.shape[0], -1)
    grad_d3 = grad(outputs=self.critic(interp3.float()), inputs=interp3, grad_outputs=self.ones,create_graph=True)[0].view(real.shape[0], -1)
    grad_penalty1 = ((grad_d1.norm(dim=1) - 1)**2).mean() * self.gp_lambda
    grad_penalty2 = ((grad_d2.norm(dim=1) - 1)**2).mean() * self.gp_lambda
    grad_penalty3 = ((grad_d3.norm(dim=1) - 1)**2).mean() * self.gp_lambda
    w_dist1 = self.critic(fake1.float()).mean() - self.critic(real.float()).mean()
    w_dist2 = self.critic(fake2.float()).mean() - self.critic(real.float()).mean()
    w_dist3 = self.critic(fake3.float()).mean() - self.critic(real.float()).mean()
    loss = w_dist1 + w_dist2 + w_dist3 + grad_penalty1 + grad_penalty2 + grad_penalty3 
    loss.backward()
    self.critic_optimizer.step()

def mlp_MIGAN2(missing_data, missing_row1,missing_row2,missing_col1,missing_col2, n=200, p_f=250, device='cuda:0', Batch_size=16, ilr=1e-3, clr=1e-3, gamma=0.2, weight=0.1, gp_lambda=10, epochs=300, MI_seed=1):
  rawdata = missing_data.copy()
  rawdata_inc1 = rawdata[np.logical_and(missing_row1,np.logical_not(missing_row2)), :].copy()
  rawdata_inc2 = rawdata[np.logical_and(missing_row2,np.logical_not(missing_row1)), :].copy()
  rawdata_inc3 = rawdata[np.logical_and(missing_row1,missing_row2), :].copy()
  cc = torch.from_numpy(rawdata.copy()[np.logical_or(missing_row1,missing_row2)==0, :])

  cc_data = Generate_data_model(cc)
  cc_data_pat1 =  Generate_data_model_2(cc, missing_col1)
  cc_data_pat2 =  Generate_data_model_2(cc, missing_col2)
  cc_data_pat3 =  Generate_data_model_2(cc, np.logical_or(missing_col1,missing_col2))

  cc_data_loader = DataLoader(cc_data, batch_size=Batch_size, shuffle=True)
  cc_data_loader1 = DataLoader(cc_data_pat1, batch_size=Batch_size, shuffle=True)
  cc_data_loader2 = DataLoader(cc_data_pat2, batch_size=Batch_size, shuffle=True)
  cc_data_loader3 = DataLoader(cc_data_pat3, batch_size=Batch_size, shuffle=True)

  imputer1 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(missing_col1)).to(device)
  imputer2 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(missing_col2)).to(device)
  imputer3 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(np.logical_or(missing_col1,missing_col2))).to(device)  

  critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)    
  imputer_params = list(imputer1.parameters()) + list(imputer2.parameters()) + list(imputer3.parameters())
  imputer_optimizer = optim.Adam(imputer_params, lr=ilr, betas=(.5, .9))
  critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
  scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
  scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
  mse_loss = torch.nn.MSELoss(reduction='mean')
  l1_loss = nn.L1Loss(reduction='mean')

  critic_updates = 0
  n_critic = 5

  for epoch in range(1,1+epochs):
    for cc in cc_data_loader:      
      cc = cc.to(device).float()                           
      batch_size = cc.shape[0]
      impu_noise = torch.empty(batch_size, p_f+1, device=device)
     
      cc_1, mask1 = next(iter(cc_data_loader1))
      cc_1 = cc_1[0:batch_size].to(device).float()
      mask1 = mask1[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data1 = imputer1(cc_1, mask1, impu_noise)
      cc_hat1 = cc_1.clone()
      cc_hat1[:,missing_col1] = imputed_data1

      cc_2, mask2 = next(iter(cc_data_loader2))
      cc_2 = cc_2[0:batch_size].to(device).float()
      mask2 = mask2[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data2 = imputer2(cc_2, mask2, impu_noise)
      cc_hat2 = cc_2.clone()
      cc_hat2[:,missing_col2] = imputed_data2

      cc_3, mask3 = next(iter(cc_data_loader3))
      cc_3 = cc_3[0:batch_size].to(device).float()
      mask3 = mask3[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data3 = imputer3(cc_3, mask3, impu_noise)
      cc_hat3 = cc_3.clone()
      cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3
      
      update_critic = CriticUpdater_model_2(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
      update_critic(cc, cc_hat1, cc_hat2, cc_hat3)
      critic_updates += 1

      if critic_updates == n_critic:
        critic_updates = 0
        reconstruction_loss = 0

        impu_noise.normal_(mean=0,std=1)
        imputed_data1 = imputer1(cc_1, mask1, impu_noise)
        cc_hat1 = cc_1.clone()
        cc_hat1[:,missing_col1] = imputed_data1
        reconstruction_loss += l1_loss(imputed_data1, cc_1[:,missing_col1].float())
        
        impu_noise.normal_(mean=0,std=1)
        imputed_data2 = imputer2(cc_2, mask2, impu_noise)
        cc_hat2 = cc_2.clone()
        cc_hat2[:,missing_col2] = imputed_data2
        reconstruction_loss += l1_loss(imputed_data2, cc_2[:,missing_col2].float())

        impu_noise.normal_(mean=0,std=1)
        imputed_data3 = imputer3(cc_3, mask3, impu_noise)
        cc_hat3 = cc_3.clone()
        cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3
        reconstruction_loss += l1_loss(imputed_data3, cc_3[:,np.logical_or(missing_col1,missing_col2)].float())

        loss = -critic(cc_hat1).mean()-critic(cc_hat2).mean()-critic(cc_hat3).mean() + weight*reconstruction_loss
        
        imputer1.zero_grad()
        imputer2.zero_grad()
        imputer3.zero_grad()
        loss.backward()
        imputer_optimizer.step()

    scheduler1.step()
    scheduler2.step()

  alist = []
  with torch.no_grad():
    imputer1.eval()
    imputer2.eval()
    imputer3.eval()
    inc_data_pat1 = torch.from_numpy(rawdata_inc1).to(device)
    inc_data_pat2 = torch.from_numpy(rawdata_inc2).to(device)
    inc_data_pat3 = torch.from_numpy(rawdata_inc3).to(device)
    mask1 = torch.from_numpy(missing_col1).to(device)
    mask2 = torch.from_numpy(missing_col2).to(device)
    mask3 = torch.from_numpy(np.logical_and(missing_col1,missing_col2)).to(device)
    for i in range(1):
      torch.manual_seed(MI_seed)
      impu_noise = torch.empty(sum(np.logical_and(missing_row1,np.logical_not(missing_row2))), p_f+1, device=device)
      imputed_data = imputer1(inc_data_pat1.float(), mask1.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc1[:,missing_col1] = imputed_data.cpu().numpy()
      # print(sum(np.isnan(imputed_data.cpu().numpy())))
      rawdata[np.logical_and(missing_row1,np.logical_not(missing_row2))] = rawdata_inc1.copy()
      
      impu_noise = torch.empty(sum(np.logical_and(missing_row2,np.logical_not(missing_row1))), p_f+1, device=device)
      imputed_data = imputer2(inc_data_pat2.float(), mask2.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc2[:,missing_col2] = imputed_data.cpu().numpy()
      # print(sum(np.isnan(imputed_data.cpu().numpy())))
      rawdata[np.logical_and(missing_row2,np.logical_not(missing_row1))] = rawdata_inc2.copy()

      impu_noise = torch.empty(sum(np.logical_and(missing_row1,missing_row2)), p_f+1, device=device)
      imputed_data = imputer3(inc_data_pat3.float(), mask3.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data.cpu().numpy()
      # print(sum(np.isnan(imputed_data.cpu().numpy())))
      rawdata[np.logical_and(missing_row1, missing_row2)] = rawdata_inc3.copy()

      alist.append(rawdata.copy())
  return alist

In [ ]:
def mlp_MIGAN2_version2(missing_data, missing_row1,missing_row2,missing_col1,missing_col2, n=200, p_f=250, device='cuda:0', Batch_size=16, ilr=1e-3, clr=1e-3, gamma=0.2, weight=0.1, gp_lambda=10, epochs=300, MI_seed=10):
  rawdata = missing_data.copy()
  rawdata_inc1 = rawdata[np.logical_and(missing_row1,np.logical_not(missing_row2)), :].copy()
  rawdata_inc2 = rawdata[np.logical_and(missing_row2,np.logical_not(missing_row1)), :].copy()
  rawdata_inc3 = rawdata[np.logical_and(missing_row1,missing_row2), :].copy()
  cc = torch.from_numpy(rawdata.copy()[np.logical_or(missing_row1,missing_row2)==0, :])

  cc_data = Generate_data_model(cc)
  cc_data_pat1 =  Generate_data_model_2(cc, missing_col1)
  cc_data_pat2 =  Generate_data_model_2(cc, missing_col2)
  cc_data_pat3 =  Generate_data_model_2(cc, np.logical_or(missing_col1,missing_col2))

  cc_data_loader = DataLoader(cc_data, batch_size=Batch_size, shuffle=True)
  cc_data_loader1 = DataLoader(cc_data_pat1, batch_size=Batch_size, shuffle=True)
  cc_data_loader2 = DataLoader(cc_data_pat2, batch_size=Batch_size, shuffle=True)
  cc_data_loader3 = DataLoader(cc_data_pat3, batch_size=Batch_size, shuffle=True)

  mse_loss = torch.nn.MSELoss(reduction='mean')
  l1_loss = nn.L1Loss(reduction='mean')
  critic_updates = 0
  n_critic = 5
  
  imputer1 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(missing_col1)).to(device) 
  critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)    
  imputer_optimizer = optim.Adam(imputer1.parameters(), lr=ilr, betas=(.5, .9))
  critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
  scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
  scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
  
  critic_updates = 0
  for epoch in range(1,1+epochs):
    for cc in cc_data_loader:      
      cc = cc.to(device).float()                           
      batch_size = cc.shape[0]
      impu_noise = torch.empty(batch_size, p_f+1, device=device)
    
      cc_1, mask1 = next(iter(cc_data_loader1))
      cc_1 = cc_1[0:batch_size].to(device).float()
      mask1 = mask1[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data1 = imputer1(cc_1, mask1, impu_noise)
      cc_hat1 = cc_1.clone()
      cc_hat1[:,missing_col1] = imputed_data1
      update_critic = CriticUpdater_model(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
      update_critic(cc, cc_hat1)
      critic_updates += 1

      if critic_updates == n_critic:
        critic_updates = 0
        reconstruction_loss = 0

        impu_noise.normal_(mean=0,std=1)
        imputed_data1 = imputer1(cc_1, mask1, impu_noise)
        cc_hat1 = cc_1.clone()
        cc_hat1[:,missing_col1] = imputed_data1
        reconstruction_loss += l1_loss(imputed_data1, cc_1[:,missing_col1].float())
        loss = -critic(cc_hat1).mean() + weight*reconstruction_loss
        imputer1.zero_grad()
        loss.backward()
        imputer_optimizer.step()
    scheduler1.step()
    scheduler2.step()
  
  imputer2 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(missing_col2)).to(device) 
  critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)
  imputer_optimizer = optim.Adam(imputer2.parameters(), lr=ilr, betas=(.5, .9))
  critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
  scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
  scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
  critic_updates = 0

  for epoch in range(1,1+epochs):
    for cc in cc_data_loader:      
      cc = cc.to(device).float()                           
      batch_size = cc.shape[0]
      impu_noise = torch.empty(batch_size, p_f+1, device=device)
    
      cc_2, mask2 = next(iter(cc_data_loader2))
      cc_2 = cc_2[0:batch_size].to(device).float()
      mask2 = mask2[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data2 = imputer2(cc_2, mask2, impu_noise)
      cc_hat2 = cc_2.clone()
      cc_hat2[:,missing_col2] = imputed_data2

      update_critic = CriticUpdater_model(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
      update_critic(cc, cc_hat2)
      critic_updates += 1

      if critic_updates == n_critic:
        critic_updates = 0
        reconstruction_loss = 0
        
        impu_noise.normal_(mean=0,std=1)
        imputed_data2 = imputer2(cc_2, mask2, impu_noise)
        cc_hat2 = cc_2.clone()
        cc_hat2[:,missing_col2] = imputed_data2
        reconstruction_loss += l1_loss(imputed_data2, cc_2[:,missing_col2].float())
        loss = -critic(cc_hat2).mean() + weight*reconstruction_loss
        imputer2.zero_grad()
        loss.backward()
        imputer_optimizer.step()
    scheduler1.step()
    scheduler2.step()
  
  imputer3 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(np.logical_or(missing_col1,missing_col2))).to(device) 
  critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)
  imputer_optimizer = optim.Adam(imputer3.parameters(), lr=ilr, betas=(.5, .9))
  critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
  scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
  scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
  critic_updates = 0

  for epoch in range(1,1+epochs):
    for cc in cc_data_loader:      
      cc = cc.to(device).float()                           
      batch_size = cc.shape[0]
      impu_noise = torch.empty(batch_size, p_f+1, device=device)
    
      cc_3, mask3 = next(iter(cc_data_loader3))
      cc_3 = cc_3[0:batch_size].to(device).float()
      mask3 = mask3[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data3 = imputer3(cc_3, mask3, impu_noise)
      cc_hat3 = cc_3.clone()
      cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3

      update_critic = CriticUpdater_model(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
      update_critic(cc, cc_hat3)
      critic_updates += 1

      if critic_updates == n_critic:
        critic_updates = 0
        reconstruction_loss = 0
        
        impu_noise.normal_(mean=0,std=1)
        imputed_data3 = imputer3(cc_3, mask3, impu_noise)
        cc_hat3 = cc_3.clone()
        cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3
        reconstruction_loss += l1_loss(imputed_data3, cc_3[:,np.logical_or(missing_col1,missing_col2)].float())
        loss = -critic(cc_hat3).mean() + weight*reconstruction_loss
        imputer3.zero_grad()
        loss.backward()
        imputer_optimizer.step()
    scheduler1.step()
    scheduler2.step()
  
  alist=[]
  with torch.no_grad():
    imputer1.eval()
    imputer2.eval()
    imputer3.eval()
    inc_data_pat1 = torch.from_numpy(rawdata_inc1).to(device)
    inc_data_pat2 = torch.from_numpy(rawdata_inc2).to(device)
    inc_data_pat3 = torch.from_numpy(rawdata_inc3).to(device)
    mask1 = torch.from_numpy(missing_col1).to(device)
    mask2 = torch.from_numpy(missing_col2).to(device)
    mask3 = torch.from_numpy(np.logical_and(missing_col1,missing_col2)).to(device)
    for i in range(1):
      torch.manual_seed(MI_seed)
      impu_noise = torch.empty(sum(np.logical_and(missing_row1,np.logical_not(missing_row2))), p_f+1, device=device)
      imputed_data = imputer1(inc_data_pat1.float(), mask1.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc1[:,missing_col1] = imputed_data.cpu().numpy()
      rawdata[np.logical_and(missing_row1,np.logical_not(missing_row2))] = rawdata_inc1.copy()
      
      impu_noise = torch.empty(sum(np.logical_and(missing_row2,np.logical_not(missing_row1))), p_f+1, device=device)
      imputed_data = imputer2(inc_data_pat2.float(), mask2.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc2[:,missing_col2] = imputed_data.cpu().numpy()
      rawdata[np.logical_and(missing_row2,np.logical_not(missing_row1))] = rawdata_inc2.copy()

      impu_noise = torch.empty(sum(np.logical_and(missing_row1,missing_row2)), p_f+1, device=device)
      imputed_data = imputer3(inc_data_pat3.float(), mask3.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data.cpu().numpy()
      rawdata[np.logical_and(missing_row1, missing_row2)] = rawdata_inc3.copy()
      
      alist.append(rawdata.copy())
  return alist

## iterative approach, column pattern 

In [ ]:
def mlp_MIGAN2_iterative(missing_data, missing_row1,missing_row2,missing_col1,missing_col2, n=200, p_f=250, device='cuda:0', Batch_size=16, ilr=1e-3, clr=1e-3, gamma=0.2, weight=0.1, gp_lambda=10, epochs=300, MI=10):
  rawdata = missing_data.copy()
  rawdata_inc1 = rawdata[np.logical_and(missing_row1,np.logical_not(missing_row2)), :].copy()
  rawdata_inc2 = rawdata[np.logical_and(missing_row2,np.logical_not(missing_row1)), :].copy()
  rawdata_inc3 = rawdata[np.logical_and(missing_row1,missing_row2), :].copy()
  cc = torch.from_numpy(rawdata.copy()[np.logical_or(missing_row1,missing_row2)==0, :])

  cc_data = Generate_data_model(cc)
  cc_data_pat1 =  Generate_data_model_2(cc, missing_col1)
  cc_data_pat2 =  Generate_data_model_2(cc, missing_col2)
  cc_data_pat3 =  Generate_data_model_2(cc, np.logical_or(missing_col1,missing_col2))

  cc_data_loader = DataLoader(cc_data, batch_size=Batch_size, shuffle=True)
  cc_data_loader1 = DataLoader(cc_data_pat1, batch_size=Batch_size, shuffle=True)
  cc_data_loader2 = DataLoader(cc_data_pat2, batch_size=Batch_size, shuffle=True)
  cc_data_loader3 = DataLoader(cc_data_pat3, batch_size=Batch_size, shuffle=True)
  
  imputer1 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(missing_col1)).to(device)
  imputer2 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(missing_col2)).to(device)
  imputer3 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(np.logical_or(missing_col1,missing_col2))).to(device)  

  critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)    
  imputer_params = list(imputer1.parameters()) + list(imputer2.parameters()) + list(imputer3.parameters())
  imputer_optimizer = optim.Adam(imputer_params, lr=ilr, betas=(.5, .9))
  critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
  scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
  scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
  mse_loss = torch.nn.MSELoss(reduction='mean')
  l1_loss = nn.L1Loss(reduction='mean')

  critic_updates = 0
  n_critic = 5

  for epoch in range(1,1+epochs):
    for cc in cc_data_loader:      
      cc = cc.to(device).float()                           
      batch_size = cc.shape[0]
      impu_noise = torch.empty(batch_size, p_f+1, device=device)
     
      cc_1, mask1 = next(iter(cc_data_loader1))
      cc_1 = cc_1[0:batch_size].to(device).float()
      mask1 = mask1[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data1 = imputer1(cc_1, mask1, impu_noise)
      cc_hat1 = cc_1.clone()
      cc_hat1[:,missing_col1] = imputed_data1

      cc_2, mask2 = next(iter(cc_data_loader2))
      cc_2 = cc_2[0:batch_size].to(device).float()
      mask2 = mask2[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data2 = imputer2(cc_2, mask2, impu_noise)
      cc_hat2 = cc_2.clone()
      cc_hat2[:,missing_col2] = imputed_data2

      cc_3, mask3 = next(iter(cc_data_loader3))
      cc_3 = cc_3[0:batch_size].to(device).float()
      mask3 = mask3[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data3 = imputer3(cc_3, mask3, impu_noise)
      cc_hat3 = cc_3.clone()
      cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3
      
      update_critic = CriticUpdater_model_2(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
      update_critic(cc, cc_hat1, cc_hat2, cc_hat3)
      critic_updates += 1

      if critic_updates == n_critic:
        critic_updates = 0
        reconstruction_loss = 0

        impu_noise.normal_(mean=0,std=1)
        imputed_data1 = imputer1(cc_1, mask1, impu_noise)
        cc_hat1 = cc_1.clone()
        cc_hat1[:,missing_col1] = imputed_data1
        reconstruction_loss += l1_loss(imputed_data1, cc_1[:,missing_col1].float())
        
        impu_noise.normal_(mean=0,std=1)
        imputed_data2 = imputer2(cc_2, mask2, impu_noise)
        cc_hat2 = cc_2.clone()
        cc_hat2[:,missing_col2] = imputed_data2
        reconstruction_loss += l1_loss(imputed_data2, cc_2[:,missing_col2].float())

        impu_noise.normal_(mean=0,std=1)
        imputed_data3 = imputer3(cc_3, mask3, impu_noise)
        cc_hat3 = cc_3.clone()
        cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3
        reconstruction_loss += l1_loss(imputed_data3, cc_3[:,np.logical_or(missing_col1,missing_col2)].float())

        loss = -critic(cc_hat1).mean()-critic(cc_hat2).mean()-critic(cc_hat3).mean() + weight*reconstruction_loss
        
        imputer1.zero_grad()
        imputer2.zero_grad()
        imputer3.zero_grad()
        loss.backward()
        imputer_optimizer.step()

    scheduler1.step()
    scheduler2.step()

  with torch.no_grad():
    imputer1.eval()
    imputer2.eval()
    imputer3.eval()
    inc_data_pat1 = torch.from_numpy(rawdata_inc1).to(device)
    inc_data_pat2 = torch.from_numpy(rawdata_inc2).to(device)
    inc_data_pat3 = torch.from_numpy(rawdata_inc3).to(device)
    mask1 = torch.from_numpy(missing_col1).to(device)
    mask2 = torch.from_numpy(missing_col2).to(device)
    mask3 = torch.from_numpy(np.logical_and(missing_col1,missing_col2)).to(device)
    for i in range(1):
      impu_noise = torch.empty(sum(np.logical_and(missing_row1,np.logical_not(missing_row2))), p_f+1, device=device)
      imputed_data = imputer1(inc_data_pat1.float(), mask1.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc1[:,missing_col1] = imputed_data.cpu().numpy()
      rawdata[np.logical_and(missing_row1,np.logical_not(missing_row2))] = rawdata_inc1.copy()
      
      impu_noise = torch.empty(sum(np.logical_and(missing_row2,np.logical_not(missing_row1))), p_f+1, device=device)
      imputed_data = imputer2(inc_data_pat2.float(), mask2.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc2[:,missing_col2] = imputed_data.cpu().numpy()
      rawdata[np.logical_and(missing_row2,np.logical_not(missing_row1))] = rawdata_inc2.copy()

      impu_noise = torch.empty(sum(np.logical_and(missing_row1,missing_row2)), p_f+1, device=device)
      imputed_data = imputer3(inc_data_pat3.float(), mask3.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data.cpu().numpy()
      rawdata[np.logical_and(missing_row1, missing_row2)] = rawdata_inc3.copy()
  
  alist = []
  for j in range(1+MI):
    rawdata_pat1 = rawdata[missing_row1, :].copy()
    train_pat1 = rawdata[np.logical_not(missing_row1), :].copy()
    train_pat1 =  Generate_data_model_2(train_pat1, missing_col1)
    train_pat1_loader1 = DataLoader(train_pat1, batch_size=Batch_size, shuffle=True)
    train_pat1_loader2 = DataLoader(train_pat1, batch_size=Batch_size, shuffle=True)
    
    imputer1 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(missing_col1)).to(device) 
    critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)    
    imputer_optimizer = optim.Adam(imputer1.parameters(), lr=ilr, betas=(.5, .9))
    critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
    scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
    scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
    
    critic_updates = 0
    for epoch in range(1,1+epochs):
      for cc, mask in train_pat1_loader1:      
        cc = cc.to(device).float()                           
        batch_size = cc.shape[0]
        impu_noise = torch.empty(batch_size, p_f+1, device=device)
      
        cc_1, mask1 = next(iter(train_pat1_loader2))
        cc_1 = cc_1[0:batch_size].to(device).float()
        mask1 = mask1[0:batch_size].to(device).float()
        impu_noise.normal_(mean=0,std=1)
        imputed_data1 = imputer1(cc_1, mask1, impu_noise)
        cc_hat1 = cc_1.clone()
        cc_hat1[:,missing_col1] = imputed_data1
        update_critic = CriticUpdater_model(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
        update_critic(cc, cc_hat1)
        critic_updates += 1

        if critic_updates == n_critic:
          critic_updates = 0
          reconstruction_loss = 0

          impu_noise.normal_(mean=0,std=1)
          imputed_data1 = imputer1(cc_1, mask1, impu_noise)
          cc_hat1 = cc_1.clone()
          cc_hat1[:,missing_col1] = imputed_data1
          reconstruction_loss += l1_loss(imputed_data1, cc_1[:,missing_col1].float())
          loss = -critic(cc_hat1).mean() + weight*reconstruction_loss
          imputer1.zero_grad()
          loss.backward()
          imputer_optimizer.step()
      scheduler1.step()
      scheduler2.step()
    
    with torch.no_grad():
      imputer1.eval()
      inc_data_pat1 = torch.from_numpy(rawdata_pat1).to(device)
      mask1 = torch.from_numpy(missing_col1).to(device)
      for i in range(1):
        torch.manual_seed(j)
        impu_noise = torch.empty(sum(missing_row1), p_f+1, device=device)
        imputed_data = imputer1(inc_data_pat1.float(), mask1.float(), impu_noise.normal_(mean=0,std=1)) 
        rawdata_pat1[:,missing_col1] = imputed_data.cpu().numpy()
        rawdata[missing_row1] = rawdata_pat1.copy()

    rawdata_pat2 = rawdata[missing_row2, :].copy()
    train_pat2 = rawdata[np.logical_not(missing_row2), :].copy()
    train_pat2 =  Generate_data_model_2(train_pat2, missing_col2)
    
    train_pat2_loader1 = DataLoader(train_pat2, batch_size=Batch_size, shuffle=True)
    train_pat2_loader2 = DataLoader(train_pat2, batch_size=Batch_size, shuffle=True)
    imputer2 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(missing_col2)).to(device) 
    critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)
    imputer_optimizer = optim.Adam(imputer2.parameters(), lr=ilr, betas=(.5, .9))
    critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
    scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
    scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
    critic_updates = 0

    for epoch in range(1,1+epochs):
      for cc, mask in train_pat2_loader1:      
        cc = cc.to(device).float()                           
        batch_size = cc.shape[0]
        impu_noise = torch.empty(batch_size, p_f+1, device=device)
      
        cc_2, mask2 = next(iter(train_pat2_loader2))
        cc_2 = cc_2[0:batch_size].to(device).float()
        mask2 = mask2[0:batch_size].to(device).float()
        impu_noise.normal_(mean=0,std=1)
        imputed_data2 = imputer2(cc_2, mask2, impu_noise)
        cc_hat2 = cc_2.clone()
        cc_hat2[:,missing_col2] = imputed_data2

        update_critic = CriticUpdater_model(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
        update_critic(cc, cc_hat2)
        critic_updates += 1

        if critic_updates == n_critic:
          critic_updates = 0
          reconstruction_loss = 0
          
          impu_noise.normal_(mean=0,std=1)
          imputed_data2 = imputer2(cc_2, mask2, impu_noise)
          cc_hat2 = cc_2.clone()
          cc_hat2[:,missing_col2] = imputed_data2
          reconstruction_loss += l1_loss(imputed_data2, cc_2[:,missing_col2].float())
          loss = -critic(cc_hat2).mean() + weight*reconstruction_loss
          imputer2.zero_grad()
          loss.backward()
          imputer_optimizer.step()
      scheduler1.step()
      scheduler2.step()

    with torch.no_grad():
      imputer2.eval()
      inc_data_pat2 = torch.from_numpy(rawdata_pat2).to(device)
      mask2 = torch.from_numpy(missing_col2).to(device)
      for i in range(1):
        torch.manual_seed(j)
        impu_noise = torch.empty(sum(missing_row2), p_f+1, device=device)
        imputed_data = imputer2(inc_data_pat2.float(), mask2.float(), impu_noise.normal_(mean=0,std=1)) 
        rawdata_pat2[:,missing_col2] = imputed_data.cpu().numpy()
        rawdata[missing_row2] = rawdata_pat2.copy()

    if j > 0:
      alist.append(rawdata.copy())
  return alist

## iterative approach, row pattern, 1

In [ ]:
def mlp_MIGAN2_iterative2(missing_data, missing_row1,missing_row2,missing_col1,missing_col2, n=200, p_f=250, device='cuda:0', Batch_size=16, ilr=1e-3, clr=1e-3, gamma=0.2, weight=0.1, gp_lambda=10, epochs=300, MI=10):
  rawdata = missing_data.copy()
  rawdata_inc1 = rawdata[np.logical_and(missing_row1,np.logical_not(missing_row2)), :].copy()
  rawdata_inc2 = rawdata[np.logical_and(missing_row2,np.logical_not(missing_row1)), :].copy()
  rawdata_inc3 = rawdata[np.logical_and(missing_row1,missing_row2), :].copy()
  cc = torch.from_numpy(rawdata.copy()[np.logical_or(missing_row1,missing_row2)==0, :])

  cc_data = Generate_data_model(cc)
  cc_data_pat1 =  Generate_data_model_2(cc, missing_col1)
  cc_data_pat2 =  Generate_data_model_2(cc, missing_col2)
  cc_data_pat3 =  Generate_data_model_2(cc, np.logical_or(missing_col1,missing_col2))

  cc_data_loader = DataLoader(cc_data, batch_size=Batch_size, shuffle=True)
  cc_data_loader1 = DataLoader(cc_data_pat1, batch_size=Batch_size, shuffle=True)
  cc_data_loader2 = DataLoader(cc_data_pat2, batch_size=Batch_size, shuffle=True)
  cc_data_loader3 = DataLoader(cc_data_pat3, batch_size=Batch_size, shuffle=True)
  
  mse_loss = torch.nn.MSELoss(reduction='mean')
  l1_loss = nn.L1Loss(reduction='mean')
  critic_updates = 0
  n_critic = 5
  
  imputer1 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(missing_col1)).to(device) 
  critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)    
  imputer_optimizer = optim.Adam(imputer1.parameters(), lr=ilr, betas=(.5, .9))
  critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
  scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
  scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
  
  critic_updates = 0
  for epoch in range(1,1+epochs):
    for cc in cc_data_loader:      
      cc = cc.to(device).float()                           
      batch_size = cc.shape[0]
      impu_noise = torch.empty(batch_size, p_f+1, device=device)
    
      cc_1, mask1 = next(iter(cc_data_loader1))
      cc_1 = cc_1[0:batch_size].to(device).float()
      mask1 = mask1[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data1 = imputer1(cc_1, mask1, impu_noise)
      cc_hat1 = cc_1.clone()
      cc_hat1[:,missing_col1] = imputed_data1
      update_critic = CriticUpdater_model(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
      update_critic(cc, cc_hat1)
      critic_updates += 1

      if critic_updates == n_critic:
        critic_updates = 0
        reconstruction_loss = 0

        impu_noise.normal_(mean=0,std=1)
        imputed_data1 = imputer1(cc_1, mask1, impu_noise)
        cc_hat1 = cc_1.clone()
        cc_hat1[:,missing_col1] = imputed_data1
        reconstruction_loss += l1_loss(imputed_data1, cc_1[:,missing_col1].float())
        loss = -critic(cc_hat1).mean() + weight*reconstruction_loss
        imputer1.zero_grad()
        loss.backward()
        imputer_optimizer.step()
    scheduler1.step()
    scheduler2.step()
  
  imputer2 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(missing_col2)).to(device) 
  critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)
  imputer_optimizer = optim.Adam(imputer2.parameters(), lr=ilr, betas=(.5, .9))
  critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
  scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
  scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
  critic_updates = 0

  for epoch in range(1,1+epochs):
    for cc in cc_data_loader:      
      cc = cc.to(device).float()                           
      batch_size = cc.shape[0]
      impu_noise = torch.empty(batch_size, p_f+1, device=device)
    
      cc_2, mask2 = next(iter(cc_data_loader2))
      cc_2 = cc_2[0:batch_size].to(device).float()
      mask2 = mask2[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data2 = imputer2(cc_2, mask2, impu_noise)
      cc_hat2 = cc_2.clone()
      cc_hat2[:,missing_col2] = imputed_data2

      update_critic = CriticUpdater_model(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
      update_critic(cc, cc_hat2)
      critic_updates += 1

      if critic_updates == n_critic:
        critic_updates = 0
        reconstruction_loss = 0
        
        impu_noise.normal_(mean=0,std=1)
        imputed_data2 = imputer2(cc_2, mask2, impu_noise)
        cc_hat2 = cc_2.clone()
        cc_hat2[:,missing_col2] = imputed_data2
        reconstruction_loss += l1_loss(imputed_data2, cc_2[:,missing_col2].float())
        loss = -critic(cc_hat2).mean() + weight*reconstruction_loss
        imputer2.zero_grad()
        loss.backward()
        imputer_optimizer.step()
    scheduler1.step()
    scheduler2.step()
  
  imputer3 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(np.logical_or(missing_col1,missing_col2))).to(device) 
  critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)
  imputer_optimizer = optim.Adam(imputer3.parameters(), lr=ilr, betas=(.5, .9))
  critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
  scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
  scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
  critic_updates = 0

  for epoch in range(1,1+epochs):
    for cc in cc_data_loader:      
      cc = cc.to(device).float()                           
      batch_size = cc.shape[0]
      impu_noise = torch.empty(batch_size, p_f+1, device=device)
    
      cc_3, mask3 = next(iter(cc_data_loader3))
      cc_3 = cc_3[0:batch_size].to(device).float()
      mask3 = mask3[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data3 = imputer3(cc_3, mask3, impu_noise)
      cc_hat3 = cc_3.clone()
      cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3

      update_critic = CriticUpdater_model(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
      update_critic(cc, cc_hat3)
      critic_updates += 1

      if critic_updates == n_critic:
        critic_updates = 0
        reconstruction_loss = 0
        
        impu_noise.normal_(mean=0,std=1)
        imputed_data3 = imputer3(cc_3, mask3, impu_noise)
        cc_hat3 = cc_3.clone()
        cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3
        reconstruction_loss += l1_loss(imputed_data3, cc_3[:,np.logical_or(missing_col1,missing_col2)].float())
        loss = -critic(cc_hat3).mean() + weight*reconstruction_loss
        imputer3.zero_grad()
        loss.backward()
        imputer_optimizer.step()
    scheduler1.step()
    scheduler2.step()
  
  with torch.no_grad():
    imputer1.eval()
    imputer2.eval()
    imputer3.eval()
    inc_data_pat1 = torch.from_numpy(rawdata_inc1).to(device)
    inc_data_pat2 = torch.from_numpy(rawdata_inc2).to(device)
    inc_data_pat3 = torch.from_numpy(rawdata_inc3).to(device)
    mask1 = torch.from_numpy(missing_col1).to(device)
    mask2 = torch.from_numpy(missing_col2).to(device)
    mask3 = torch.from_numpy(np.logical_and(missing_col1,missing_col2)).to(device)
    for i in range(1):
      impu_noise = torch.empty(sum(np.logical_and(missing_row1,np.logical_not(missing_row2))), p_f+1, device=device)
      imputed_data = imputer1(inc_data_pat1.float(), mask1.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc1[:,missing_col1] = imputed_data.cpu().numpy()
      rawdata[np.logical_and(missing_row1,np.logical_not(missing_row2))] = rawdata_inc1.copy()
      
      impu_noise = torch.empty(sum(np.logical_and(missing_row2,np.logical_not(missing_row1))), p_f+1, device=device)
      imputed_data = imputer2(inc_data_pat2.float(), mask2.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc2[:,missing_col2] = imputed_data.cpu().numpy()
      rawdata[np.logical_and(missing_row2,np.logical_not(missing_row1))] = rawdata_inc2.copy()

      impu_noise = torch.empty(sum(np.logical_and(missing_row1,missing_row2)), p_f+1, device=device)
      imputed_data = imputer3(inc_data_pat3.float(), mask3.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data.cpu().numpy()
      rawdata[np.logical_and(missing_row1, missing_row2)] = rawdata_inc3.copy()
  
  alist = []
  for j in range(1+MI):
    rawdata_pat1 = rawdata[np.logical_and(missing_row1,missing_row2==False),:].copy()
    train_pat1 = rawdata[np.logical_or(missing_row1==False,missing_row2),:].copy()
    train_pat1 = Generate_data_model_2(train_pat1, missing_col1)
    train_pat1_loader1 = DataLoader(train_pat1, batch_size=Batch_size, shuffle=True)
    train_pat1_loader2 = DataLoader(train_pat1, batch_size=Batch_size, shuffle=True)
    
    imputer1 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(missing_col1)).to(device) 
    critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)    
    imputer_optimizer = optim.Adam(imputer1.parameters(), lr=ilr, betas=(.5, .9))
    critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
    scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
    scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
    
    critic_updates = 0
    for epoch in range(1,1+epochs):
      for cc, mask in train_pat1_loader1:      
        cc = cc.to(device).float()                           
        batch_size = cc.shape[0]
        impu_noise = torch.empty(batch_size, p_f+1, device=device)
      
        cc_1, mask1 = next(iter(train_pat1_loader2))
        cc_1 = cc_1[0:batch_size].to(device).float()
        mask1 = mask1[0:batch_size].to(device).float()
        impu_noise.normal_(mean=0,std=1)
        imputed_data1 = imputer1(cc_1, mask1, impu_noise)
        cc_hat1 = cc_1.clone()
        cc_hat1[:,missing_col1] = imputed_data1
        update_critic = CriticUpdater_model(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
        update_critic(cc, cc_hat1)
        critic_updates += 1

        if critic_updates == n_critic:
          critic_updates = 0
          reconstruction_loss = 0

          impu_noise.normal_(mean=0,std=1)
          imputed_data1 = imputer1(cc_1, mask1, impu_noise)
          cc_hat1 = cc_1.clone()
          cc_hat1[:,missing_col1] = imputed_data1
          reconstruction_loss += l1_loss(imputed_data1, cc_1[:,missing_col1].float())
          loss = -critic(cc_hat1).mean() + weight*reconstruction_loss
          imputer1.zero_grad()
          loss.backward()
          imputer_optimizer.step()
      scheduler1.step()
      scheduler2.step()
    
    with torch.no_grad():
      imputer1.eval()
      inc_data_pat1 = torch.from_numpy(rawdata_pat1).to(device)
      mask1 = torch.from_numpy(missing_col1).to(device)
      for i in range(1):
        torch.manual_seed(j)
        impu_noise = torch.empty(sum(np.logical_and(missing_row1,missing_row2==False)), p_f+1, device=device)
        imputed_data = imputer1(inc_data_pat1.float(), mask1.float(), impu_noise.normal_(mean=0,std=1)) 
        rawdata_pat1[:,missing_col1] = imputed_data.cpu().numpy()
        rawdata[np.logical_and(missing_row1,missing_row2==False)] = rawdata_pat1.copy()

    rawdata_pat2 = rawdata[np.logical_and(missing_row1==False,missing_row2),:].copy()
    train_pat2 = rawdata[np.logical_or(missing_row1,missing_row2==False),:].copy()
    train_pat2 =  Generate_data_model_2(train_pat2, missing_col2)
    
    train_pat2_loader1 = DataLoader(train_pat2, batch_size=Batch_size, shuffle=True)
    train_pat2_loader2 = DataLoader(train_pat2, batch_size=Batch_size, shuffle=True)
    imputer2 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(missing_col2)).to(device) 
    critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)
    imputer_optimizer = optim.Adam(imputer2.parameters(), lr=ilr, betas=(.5, .9))
    critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
    scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
    scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
    critic_updates = 0

    for epoch in range(1,1+epochs):
      for cc, mask in train_pat2_loader1:      
        cc = cc.to(device).float()                           
        batch_size = cc.shape[0]
        impu_noise = torch.empty(batch_size, p_f+1, device=device)
      
        cc_2, mask2 = next(iter(train_pat2_loader2))
        cc_2 = cc_2[0:batch_size].to(device).float()
        mask2 = mask2[0:batch_size].to(device).float()
        impu_noise.normal_(mean=0,std=1)
        imputed_data2 = imputer2(cc_2, mask2, impu_noise)
        cc_hat2 = cc_2.clone()
        cc_hat2[:,missing_col2] = imputed_data2

        update_critic = CriticUpdater_model(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
        update_critic(cc, cc_hat2)
        critic_updates += 1

        if critic_updates == n_critic:
          critic_updates = 0
          reconstruction_loss = 0
          
          impu_noise.normal_(mean=0,std=1)
          imputed_data2 = imputer2(cc_2, mask2, impu_noise)
          cc_hat2 = cc_2.clone()
          cc_hat2[:,missing_col2] = imputed_data2
          reconstruction_loss += l1_loss(imputed_data2, cc_2[:,missing_col2].float())
          loss = -critic(cc_hat2).mean() + weight*reconstruction_loss
          imputer2.zero_grad()
          loss.backward()
          imputer_optimizer.step()
      scheduler1.step()
      scheduler2.step()

    with torch.no_grad():
      imputer2.eval()
      inc_data_pat2 = torch.from_numpy(rawdata_pat2).to(device)
      mask2 = torch.from_numpy(missing_col2).to(device)
      for i in range(1):
        torch.manual_seed(j)
        impu_noise = torch.empty(sum(np.logical_and(missing_row1==False,missing_row2)), p_f+1, device=device)
        imputed_data = imputer2(inc_data_pat2.float(), mask2.float(), impu_noise.normal_(mean=0,std=1)) 
        rawdata_pat2[:,missing_col2] = imputed_data.cpu().numpy()
        rawdata[np.logical_and(missing_row1==False,missing_row2)] = rawdata_pat2.copy()
    
    rawdata_pat3 = rawdata[np.logical_and(missing_row1,missing_row2),:].copy()
    train_pat3 = rawdata[np.logical_or(missing_row1==False,missing_row2==False),:].copy()
    train_pat3 =  Generate_data_model_2(train_pat3, np.logical_or(missing_col1,missing_col2))
    
    train_pat3_loader1 = DataLoader(train_pat3, batch_size=Batch_size, shuffle=True)
    train_pat3_loader2 = DataLoader(train_pat3, batch_size=Batch_size, shuffle=True)
    imputer3 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(np.logical_or(missing_col1,missing_col2))).to(device) 
    critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)
    imputer_optimizer = optim.Adam(imputer3.parameters(), lr=ilr, betas=(.5, .9))
    critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
    scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
    scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
    critic_updates = 0

    for epoch in range(1,1+epochs):
      for cc, mask in train_pat3_loader1:      
        cc = cc.to(device).float()                           
        batch_size = cc.shape[0]
        impu_noise = torch.empty(batch_size, p_f+1, device=device)
      
        cc_3, mask3 = next(iter(train_pat3_loader2))
        cc_3 = cc_3[0:batch_size].to(device).float()
        mask3 = mask3[0:batch_size].to(device).float()
        impu_noise.normal_(mean=0,std=1)
        imputed_data3 = imputer3(cc_3, mask3, impu_noise)
        cc_hat3 = cc_3.clone()
        cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3

        update_critic = CriticUpdater_model(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
        update_critic(cc, cc_hat3)
        critic_updates += 1

        if critic_updates == n_critic:
          critic_updates = 0
          reconstruction_loss = 0
          
          impu_noise.normal_(mean=0,std=1)
          imputed_data3 = imputer3(cc_3, mask3, impu_noise)
          cc_hat3 = cc_3.clone()
          cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3
          reconstruction_loss += l1_loss(imputed_data3, cc_3[:,np.logical_or(missing_col1,missing_col2)].float())
          loss = -critic(cc_hat3).mean() + weight*reconstruction_loss
          imputer3.zero_grad()
          loss.backward()
          imputer_optimizer.step()
      scheduler1.step()
      scheduler2.step()

    with torch.no_grad():
      imputer3.eval()
      inc_data_pat3 = torch.from_numpy(rawdata_pat3).to(device)
      mask3 = torch.from_numpy(np.logical_or(missing_col1,missing_col2)).to(device)
      for i in range(1):
        torch.manual_seed(j)
        impu_noise = torch.empty(sum(np.logical_and(missing_row1,missing_row2)), p_f+1, device=device)
        imputed_data = imputer3(inc_data_pat3.float(), mask3.float(), impu_noise.normal_(mean=0,std=1)) 
        rawdata_pat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data.cpu().numpy()
        rawdata[np.logical_and(missing_row1,missing_row2)] = rawdata_pat3.copy()

    if j > 0:
      alist.append(rawdata.copy())
  return alist

In [ ]:
def mlp_MIGAN2_iterative4(missing_data, missing_row1,missing_row2,missing_col1,missing_col2, n=200, p_f=250, device='cuda:0', Batch_size=16, ilr=1e-3, clr=1e-3, gamma=0.2, weight=0.1, gp_lambda=10, epochs=300, MI=10):
  rawdata = missing_data.copy()
  rawdata_pat1 = rawdata[np.logical_and(missing_row1,np.logical_not(missing_row2)), :].copy()
  rawdata_pat2 = rawdata[np.logical_and(missing_row2,np.logical_not(missing_row1)), :].copy()
  rawdata_pat3 = rawdata[np.logical_and(missing_row1,missing_row2), :].copy()
  cc = torch.from_numpy(rawdata.copy()[np.logical_or(missing_row1,missing_row2)==0, :])

  cc_data = Generate_data_model(cc)
  cc_data_pat1 =  Generate_data_model_2(cc, missing_col1)
  cc_data_pat2 =  Generate_data_model_2(cc, missing_col2)
  cc_data_pat3 =  Generate_data_model_2(cc, np.logical_or(missing_col1,missing_col2))

  cc_data_loader = DataLoader(cc_data, batch_size=Batch_size, shuffle=True)
  cc_data_loader1 = DataLoader(cc_data_pat1, batch_size=Batch_size, shuffle=True)
  cc_data_loader2 = DataLoader(cc_data_pat2, batch_size=Batch_size, shuffle=True)
  cc_data_loader3 = DataLoader(cc_data_pat3, batch_size=Batch_size, shuffle=True)
  
  imputer1 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(missing_col1)).to(device) 
  critic1 = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)    
  imputer_optimizer1 = optim.Adam(imputer1.parameters(), lr=ilr, betas=(.5, .9))
  critic_optimizer1 = optim.Adam(critic1.parameters(), lr=clr, betas=(.5, .9))
  scheduler1_1 = optim.lr_scheduler.StepLR(imputer_optimizer1, step_size=10, gamma=gamma)
  scheduler1_2 = optim.lr_scheduler.StepLR(critic_optimizer1, step_size=10, gamma=gamma)

  imputer2 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(missing_col2)).to(device) 
  critic2 = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)
  imputer_optimizer2 = optim.Adam(imputer2.parameters(), lr=ilr, betas=(.5, .9))
  critic_optimizer2 = optim.Adam(critic2.parameters(), lr=clr, betas=(.5, .9))
  scheduler2_1 = optim.lr_scheduler.StepLR(imputer_optimizer2, step_size=10, gamma=gamma)
  scheduler2_2 = optim.lr_scheduler.StepLR(critic_optimizer2, step_size=10, gamma=gamma)
  
  imputer3 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(np.logical_or(missing_col1,missing_col2))).to(device) 
  critic3 = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)
  imputer_optimizer3 = optim.Adam(imputer3.parameters(), lr=ilr, betas=(.5, .9))
  critic_optimizer3 = optim.Adam(critic3.parameters(), lr=clr, betas=(.5, .9))
  scheduler3_1 = optim.lr_scheduler.StepLR(imputer_optimizer3, step_size=10, gamma=gamma)
  scheduler3_2 = optim.lr_scheduler.StepLR(critic_optimizer3, step_size=10, gamma=gamma)
  mse_loss = torch.nn.MSELoss(reduction='mean')
  l1_loss = nn.L1Loss(reduction='mean')

  critic_updates = 0
  n_critic = 5

  for epoch in range(1,1+epochs):
    for cc in cc_data_loader:      
      cc = cc.to(device).float()                           
      batch_size = cc.shape[0]
      impu_noise = torch.empty(batch_size, p_f+1, device=device)
     
      cc_1, mask1 = next(iter(cc_data_loader1))
      cc_1 = cc_1[0:batch_size].to(device).float()
      mask1 = mask1[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data1 = imputer1(cc_1, mask1, impu_noise)
      cc_hat1 = cc_1.clone()
      cc_hat1[:,missing_col1] = imputed_data1
      update_critic = CriticUpdater_model(critic1, critic_optimizer1, batch_size, device=device, gp_lambda=gp_lambda)
      update_critic(cc, cc_hat1)
      critic_updates += 1

      if critic_updates == n_critic:
        critic_updates = 0
        reconstruction_loss = 0

        impu_noise.normal_(mean=0,std=1)
        imputed_data1 = imputer1(cc_1, mask1, impu_noise)
        cc_hat1 = cc_1.clone()
        cc_hat1[:,missing_col1] = imputed_data1
        reconstruction_loss += l1_loss(imputed_data1, cc_1[:,missing_col1].float())
        loss = -critic1(cc_hat1).mean() + weight*reconstruction_loss
        imputer1.zero_grad()
        loss.backward()
        imputer_optimizer1.step()
    scheduler1_1.step()
    scheduler1_2.step()
  
  with torch.no_grad():
    imputer1.eval()
    inc_data_pat1 = torch.from_numpy(rawdata_pat1).to(device)
    mask1 = torch.from_numpy(missing_col1).to(device)
    for i in range(1):
      impu_noise = torch.empty(sum(np.logical_and(missing_row1,missing_row2==False)), p_f+1, device=device)
      imputed_data = imputer1(inc_data_pat1.float(), mask1.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_pat1[:,missing_col1] = imputed_data.cpu().numpy()
      rawdata[np.logical_and(missing_row1,missing_row2==False)] = rawdata_pat1.copy()
   
  for epoch in range(1,1+epochs):
    for cc in cc_data_loader:      
      cc = cc.to(device).float()                           
      batch_size = cc.shape[0]
      impu_noise = torch.empty(batch_size, p_f+1, device=device)
     
      cc_2, mask2 = next(iter(cc_data_loader2))
      cc_2 = cc_2[0:batch_size].to(device).float()
      mask2 = mask2[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data2 = imputer2(cc_2, mask2, impu_noise)
      cc_hat2 = cc_2.clone()
      cc_hat2[:,missing_col2] = imputed_data2

      update_critic = CriticUpdater_model(critic2, critic_optimizer2, batch_size, device=device, gp_lambda=gp_lambda)
      update_critic(cc, cc_hat2)
      critic_updates += 1

      if critic_updates == n_critic:
        critic_updates = 0
        reconstruction_loss = 0
        
        impu_noise.normal_(mean=0,std=1)
        imputed_data2 = imputer2(cc_2, mask2, impu_noise)
        cc_hat2 = cc_2.clone()
        cc_hat2[:,missing_col2] = imputed_data2
        reconstruction_loss += l1_loss(imputed_data2, cc_2[:,missing_col2].float())
        loss = -critic2(cc_hat2).mean() + weight*reconstruction_loss
        imputer2.zero_grad()
        loss.backward()
        imputer_optimizer2.step()
    scheduler2_1.step()
    scheduler2_2.step()

  with torch.no_grad():
    imputer2.eval()
    inc_data_pat2 = torch.from_numpy(rawdata_pat2).to(device)
    mask2 = torch.from_numpy(missing_col2).to(device)
    for i in range(1):
      impu_noise = torch.empty(sum(np.logical_and(missing_row1==False,missing_row2)), p_f+1, device=device)
      imputed_data = imputer2(inc_data_pat2.float(), mask2.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_pat2[:,missing_col2] = imputed_data.cpu().numpy()
      rawdata[np.logical_and(missing_row1==False,missing_row2)] = rawdata_pat2.copy()
  
  for epoch in range(1,1+epochs):
    for cc in cc_data_loader:      
      cc = cc.to(device).float()                           
      batch_size = cc.shape[0]
      impu_noise = torch.empty(batch_size, p_f+1, device=device)
     
      cc_3, mask3 = next(iter(cc_data_loader3))
      cc_3 = cc_3[0:batch_size].to(device).float()
      mask3 = mask3[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data3 = imputer3(cc_3, mask3, impu_noise)
      cc_hat3 = cc_3.clone()
      cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3

      update_critic = CriticUpdater_model(critic3, critic_optimizer3, batch_size, device=device, gp_lambda=gp_lambda)
      update_critic(cc, cc_hat3)
      critic_updates += 1

      if critic_updates == n_critic:
        critic_updates = 0
        reconstruction_loss = 0
        
        impu_noise.normal_(mean=0,std=1)
        imputed_data3 = imputer3(cc_3, mask3, impu_noise)
        cc_hat3 = cc_3.clone()
        cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3
        reconstruction_loss += l1_loss(imputed_data3, cc_3[:,np.logical_or(missing_col1,missing_col2)].float())
        loss = -critic3(cc_hat3).mean() + weight*reconstruction_loss
        imputer3.zero_grad()
        loss.backward()
        imputer_optimizer3.step()
    scheduler3_1.step()
    scheduler3_2.step()

  with torch.no_grad():
    imputer3.eval()
    inc_data_pat3 = torch.from_numpy(rawdata_pat3).to(device)
    mask3 = torch.from_numpy(np.logical_or(missing_col1,missing_col2)).to(device)
    for i in range(1):
      impu_noise = torch.empty(sum(np.logical_and(missing_row1,missing_row2)), p_f+1, device=device)
      imputed_data = imputer3(inc_data_pat3.float(), mask3.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_pat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data.cpu().numpy()
      rawdata[np.logical_and(missing_row1,missing_row2)] = rawdata_pat3.copy()


  alist = []
  for j in range(1+MI):
    train_pat1 = rawdata[np.logical_or(missing_row1==False,missing_row2),:].copy()
    train_pat1 = Generate_data_model_2(train_pat1, missing_col1)
    train_pat1_loader1 = DataLoader(train_pat1, batch_size=Batch_size, shuffle=True)
    train_pat1_loader2 = DataLoader(train_pat1, batch_size=Batch_size, shuffle=True)
    imputer_optimizer1 = optim.Adam(imputer1.parameters(), lr=ilr/10, betas=(.5, .9))
    critic_optimizer1 = optim.Adam(critic1.parameters(), lr=clr/10, betas=(.5, .9))
    scheduler1_1 = optim.lr_scheduler.StepLR(imputer_optimizer1, step_size=10, gamma=gamma)
    scheduler1_2 = optim.lr_scheduler.StepLR(critic_optimizer1, step_size=10, gamma=gamma)

    critic_updates = 0
    imputer1.train()
    for epoch in range(1,1+epochs):
      for cc, mask in train_pat1_loader1:      
        cc = cc.to(device).float()                           
        batch_size = cc.shape[0]
        impu_noise = torch.empty(batch_size, p_f+1, device=device)
      
        cc_1, mask1 = next(iter(train_pat1_loader2))
        cc_1 = cc_1[0:batch_size].to(device).float()
        mask1 = mask1[0:batch_size].to(device).float()
        impu_noise.normal_(mean=0,std=1)
        imputed_data1 = imputer1(cc_1, mask1, impu_noise)
        cc_hat1 = cc_1.clone()
        cc_hat1[:,missing_col1] = imputed_data1
        update_critic = CriticUpdater_model(critic1, critic_optimizer1, batch_size, device=device, gp_lambda=gp_lambda)
        update_critic(cc, cc_hat1)
        critic_updates += 1

        if critic_updates == n_critic:
          critic_updates = 0
          reconstruction_loss = 0

          impu_noise.normal_(mean=0,std=1)
          imputed_data1 = imputer1(cc_1, mask1, impu_noise)
          cc_hat1 = cc_1.clone()
          cc_hat1[:,missing_col1] = imputed_data1
          reconstruction_loss += l1_loss(imputed_data1, cc_1[:,missing_col1].float())
          loss = -critic1(cc_hat1).mean() + weight*reconstruction_loss
          imputer1.zero_grad()
          loss.backward()
          imputer_optimizer1.step()
      scheduler1_1.step()
      scheduler1_2.step()
    
    with torch.no_grad():
      imputer1.eval()
      inc_data_pat1 = torch.from_numpy(rawdata_pat1).to(device)
      mask1 = torch.from_numpy(missing_col1).to(device)
      for i in range(1):
        torch.manual_seed(j)
        impu_noise = torch.empty(sum(np.logical_and(missing_row1,missing_row2==False)), p_f+1, device=device)
        imputed_data = imputer1(inc_data_pat1.float(), mask1.float(), impu_noise.normal_(mean=0,std=1)) 
        rawdata_pat1[:,missing_col1] = imputed_data.cpu().numpy()
        rawdata[np.logical_and(missing_row1,missing_row2==False)] = rawdata_pat1.copy()

    train_pat2 = rawdata[np.logical_or(missing_row1,missing_row2==False),:].copy()
    train_pat2 =  Generate_data_model_2(train_pat2, missing_col2)
    train_pat2_loader1 = DataLoader(train_pat2, batch_size=Batch_size, shuffle=True)
    train_pat2_loader2 = DataLoader(train_pat2, batch_size=Batch_size, shuffle=True)
    imputer_optimizer2 = optim.Adam(imputer2.parameters(), lr=ilr/10, betas=(.5, .9))
    critic_optimizer2 = optim.Adam(critic2.parameters(), lr=clr/10, betas=(.5, .9))
    scheduler2_1 = optim.lr_scheduler.StepLR(imputer_optimizer2, step_size=10, gamma=gamma)
    scheduler2_2 = optim.lr_scheduler.StepLR(critic_optimizer2, step_size=10, gamma=gamma)

    critic_updates = 0
    imputer2.train()
    for epoch in range(1,1+epochs):
      for cc, mask in train_pat2_loader1:      
        cc = cc.to(device).float()                           
        batch_size = cc.shape[0]
        impu_noise = torch.empty(batch_size, p_f+1, device=device)
      
        cc_2, mask2 = next(iter(train_pat2_loader2))
        cc_2 = cc_2[0:batch_size].to(device).float()
        mask2 = mask2[0:batch_size].to(device).float()
        impu_noise.normal_(mean=0,std=1)
        imputed_data2 = imputer2(cc_2, mask2, impu_noise)
        cc_hat2 = cc_2.clone()
        cc_hat2[:,missing_col2] = imputed_data2

        update_critic = CriticUpdater_model(critic2, critic_optimizer2, batch_size, device=device, gp_lambda=gp_lambda)
        update_critic(cc, cc_hat2)
        critic_updates += 1

        if critic_updates == n_critic:
          critic_updates = 0
          reconstruction_loss = 0
          
          impu_noise.normal_(mean=0,std=1)
          imputed_data2 = imputer2(cc_2, mask2, impu_noise)
          cc_hat2 = cc_2.clone()
          cc_hat2[:,missing_col2] = imputed_data2
          reconstruction_loss += l1_loss(imputed_data2, cc_2[:,missing_col2].float())
          loss = -critic2(cc_hat2).mean() + weight*reconstruction_loss
          imputer2.zero_grad()
          loss.backward()
          imputer_optimizer2.step()
      scheduler2_1.step()
      scheduler2_2.step()

    with torch.no_grad():
      imputer2.eval()
      inc_data_pat2 = torch.from_numpy(rawdata_pat2).to(device)
      mask2 = torch.from_numpy(missing_col2).to(device)
      for i in range(1):
        torch.manual_seed(j)
        impu_noise = torch.empty(sum(np.logical_and(missing_row1==False,missing_row2)), p_f+1, device=device)
        imputed_data = imputer2(inc_data_pat2.float(), mask2.float(), impu_noise.normal_(mean=0,std=1)) 
        rawdata_pat2[:,missing_col2] = imputed_data.cpu().numpy()
        rawdata[np.logical_and(missing_row1==False,missing_row2)] = rawdata_pat2.copy()
    
    train_pat3 = rawdata[np.logical_or(missing_row1==False,missing_row2==False),:].copy()
    train_pat3 =  Generate_data_model_2(train_pat3, np.logical_or(missing_col1,missing_col2))
    train_pat3_loader1 = DataLoader(train_pat3, batch_size=Batch_size, shuffle=True)
    train_pat3_loader2 = DataLoader(train_pat3, batch_size=Batch_size, shuffle=True)
    imputer_optimizer3 = optim.Adam(imputer3.parameters(), lr=ilr/10, betas=(.5, .9))
    critic_optimizer3 = optim.Adam(critic3.parameters(), lr=clr/10, betas=(.5, .9))
    scheduler3_1 = optim.lr_scheduler.StepLR(imputer_optimizer3, step_size=10, gamma=gamma)
    scheduler3_2 = optim.lr_scheduler.StepLR(critic_optimizer3, step_size=10, gamma=gamma)

    critic_updates = 0
    imputer3.train()
    for epoch in range(1,1+epochs):
      for cc, mask in train_pat3_loader1:      
        cc = cc.to(device).float()                           
        batch_size = cc.shape[0]
        impu_noise = torch.empty(batch_size, p_f+1, device=device)
      
        cc_3, mask3 = next(iter(train_pat3_loader2))
        cc_3 = cc_3[0:batch_size].to(device).float()
        mask3 = mask3[0:batch_size].to(device).float()
        impu_noise.normal_(mean=0,std=1)
        imputed_data3 = imputer3(cc_3, mask3, impu_noise)
        cc_hat3 = cc_3.clone()
        cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3

        update_critic = CriticUpdater_model(critic3, critic_optimizer3, batch_size, device=device, gp_lambda=gp_lambda)
        update_critic(cc, cc_hat3)
        critic_updates += 1

        if critic_updates == n_critic:
          critic_updates = 0
          reconstruction_loss = 0
          
          impu_noise.normal_(mean=0,std=1)
          imputed_data3 = imputer3(cc_3, mask3, impu_noise)
          cc_hat3 = cc_3.clone()
          cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3
          reconstruction_loss += l1_loss(imputed_data3, cc_3[:,np.logical_or(missing_col1,missing_col2)].float())
          loss = -critic3(cc_hat3).mean() + weight*reconstruction_loss
          imputer3.zero_grad()
          loss.backward()
          imputer_optimizer3.step()
      scheduler3_1.step()
      scheduler3_2.step()

    with torch.no_grad():
      imputer3.eval()
      inc_data_pat3 = torch.from_numpy(rawdata_pat3).to(device)
      mask3 = torch.from_numpy(np.logical_or(missing_col1,missing_col2)).to(device)
      for i in range(1):
        torch.manual_seed(j)
        impu_noise = torch.empty(sum(np.logical_and(missing_row1,missing_row2)), p_f+1, device=device)
        imputed_data = imputer3(inc_data_pat3.float(), mask3.float(), impu_noise.normal_(mean=0,std=1)) 
        rawdata_pat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data.cpu().numpy()
        rawdata[np.logical_and(missing_row1,missing_row2)] = rawdata_pat3.copy()

    if j > 0:
      alist.append(rawdata.copy())
  return alist

## iterative approach, row pattern, 2, joint training

In [ ]:
class CriticUpdater_model_3:
  def __init__(self, critic1, critic2, critic3, critic_optimizer, batch_size1, batch_size2, batch_size3, gp_lambda=10, device='cuda:0'):
    self.critic1 = critic1
    self.critic2 = critic2
    self.critic3 = critic3
    self.critic_optimizer = critic_optimizer
    self.gp_lambda = gp_lambda
    # Interpolation coefficient
    self.eps1 = torch.empty(batch_size1, 1,device=device)
    self.eps2 = torch.empty(batch_size2, 1,device=device)
    self.eps3 = torch.empty(batch_size3, 1,device=device)
    # For computing the gradient penalty
    self.ones1 = torch.ones(batch_size1, 1).to(device)
    self.ones2 = torch.ones(batch_size2, 1).to(device)
    self.ones3 = torch.ones(batch_size3, 1).to(device)

  def __call__(self, real1, real2, real3, fake1, fake2, fake3, device='cuda:0'):
    real1 = real1.detach()
    real2 = real2.detach()
    real3 = real3.detach()
    fake1 = fake1.detach()
    fake2 = fake2.detach()
    fake3 = fake3.detach()
    self.critic1.zero_grad()
    self.critic2.zero_grad()
    self.critic3.zero_grad()
    self.eps1.uniform_(0, 1)
    interp1 = (self.eps1 * real1 + (1 - self.eps1) * fake1).to(device).requires_grad_()
    self.eps2.uniform_(0, 1)
    interp2 = (self.eps2 * real2 + (1 - self.eps2) * fake2).to(device).requires_grad_()
    self.eps3.uniform_(0, 1)
    interp3 = (self.eps3 * real3 + (1 - self.eps3) * fake3).to(device).requires_grad_()
    grad_d1 = grad(outputs=self.critic1(interp1.float()), inputs=interp1, grad_outputs=self.ones1,create_graph=True)[0].view(real1.shape[0], -1)  #grad_outputs=self.ones,create_graph=True
    grad_d2 = grad(outputs=self.critic2(interp2.float()), inputs=interp2, grad_outputs=self.ones2,create_graph=True)[0].view(real2.shape[0], -1)
    grad_d3 = grad(outputs=self.critic3(interp3.float()), inputs=interp3, grad_outputs=self.ones3,create_graph=True)[0].view(real3.shape[0], -1)
    grad_penalty1 = ((grad_d1.norm(dim=1) - 1)**2).mean() * self.gp_lambda
    grad_penalty2 = ((grad_d2.norm(dim=1) - 1)**2).mean() * self.gp_lambda
    grad_penalty3 = ((grad_d3.norm(dim=1) - 1)**2).mean() * self.gp_lambda
    w_dist1 = self.critic1(fake1.float()).mean() - self.critic1(real1.float()).mean()
    w_dist2 = self.critic2(fake2.float()).mean() - self.critic2(real2.float()).mean()
    w_dist3 = self.critic3(fake3.float()).mean() - self.critic3(real3.float()).mean()
    loss = w_dist1 + w_dist2 + w_dist3 + grad_penalty1 + grad_penalty2 + grad_penalty3 
    loss.backward()
    self.critic_optimizer.step()

In [ ]:
def mlp_MIGAN2_iterative3(missing_data, missing_row1,missing_row2,missing_col1,missing_col2, n=200, p_f=250, device='cuda:0', Batch_size=16, ilr=1e-3, clr=1e-3, gamma=0.2, weight=0.1, gp_lambda=10, epochs=300, MI=10):
  rawdata = missing_data.copy()
  rawdata_inc1 = rawdata[np.logical_and(missing_row1,np.logical_not(missing_row2)), :].copy()
  rawdata_inc2 = rawdata[np.logical_and(missing_row2,np.logical_not(missing_row1)), :].copy()
  rawdata_inc3 = rawdata[np.logical_and(missing_row1,missing_row2), :].copy()
  cc = torch.from_numpy(rawdata.copy()[np.logical_or(missing_row1,missing_row2)==0, :])

  cc_data = Generate_data_model(cc)
  cc_data_pat1 =  Generate_data_model_2(cc, missing_col1)
  cc_data_pat2 =  Generate_data_model_2(cc, missing_col2)
  cc_data_pat3 =  Generate_data_model_2(cc, np.logical_or(missing_col1,missing_col2))

  cc_data_loader = DataLoader(cc_data, batch_size=Batch_size, shuffle=True)
  cc_data_loader1 = DataLoader(cc_data_pat1, batch_size=Batch_size, shuffle=True)
  cc_data_loader2 = DataLoader(cc_data_pat2, batch_size=Batch_size, shuffle=True)
  cc_data_loader3 = DataLoader(cc_data_pat3, batch_size=Batch_size, shuffle=True)
  
  imputer1 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(missing_col1)).to(device)
  imputer2 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(missing_col2)).to(device)
  imputer3 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(np.logical_or(missing_col1,missing_col2))).to(device)  

  critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)    
  imputer_params = list(imputer1.parameters()) + list(imputer2.parameters()) + list(imputer3.parameters())
  imputer_optimizer = optim.Adam(imputer_params, lr=ilr, betas=(.5, .9))
  critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
  scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
  scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
  mse_loss = torch.nn.MSELoss(reduction='mean')
  l1_loss = nn.L1Loss(reduction='mean')

  critic_updates = 0
  n_critic = 5

  for epoch in range(1,1+epochs):
    for cc in cc_data_loader:      
      cc = cc.to(device).float()                           
      batch_size = cc.shape[0]
      impu_noise = torch.empty(batch_size, p_f+1, device=device)
     
      cc_1, mask1 = next(iter(cc_data_loader1))
      cc_1 = cc_1[0:batch_size].to(device).float()
      mask1 = mask1[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data1 = imputer1(cc_1, mask1, impu_noise)
      cc_hat1 = cc_1.clone()
      cc_hat1[:,missing_col1] = imputed_data1

      cc_2, mask2 = next(iter(cc_data_loader2))
      cc_2 = cc_2[0:batch_size].to(device).float()
      mask2 = mask2[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data2 = imputer2(cc_2, mask2, impu_noise)
      cc_hat2 = cc_2.clone()
      cc_hat2[:,missing_col2] = imputed_data2

      cc_3, mask3 = next(iter(cc_data_loader3))
      cc_3 = cc_3[0:batch_size].to(device).float()
      mask3 = mask3[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data3 = imputer3(cc_3, mask3, impu_noise)
      cc_hat3 = cc_3.clone()
      cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3
      
      update_critic = CriticUpdater_model_2(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
      update_critic(cc, cc_hat1, cc_hat2, cc_hat3)
      critic_updates += 1

      if critic_updates == n_critic:
        critic_updates = 0
        reconstruction_loss = 0

        impu_noise.normal_(mean=0,std=1)
        imputed_data1 = imputer1(cc_1, mask1, impu_noise)
        cc_hat1 = cc_1.clone()
        cc_hat1[:,missing_col1] = imputed_data1
        reconstruction_loss += l1_loss(imputed_data1, cc_1[:,missing_col1].float())
        
        impu_noise.normal_(mean=0,std=1)
        imputed_data2 = imputer2(cc_2, mask2, impu_noise)
        cc_hat2 = cc_2.clone()
        cc_hat2[:,missing_col2] = imputed_data2
        reconstruction_loss += l1_loss(imputed_data2, cc_2[:,missing_col2].float())

        impu_noise.normal_(mean=0,std=1)
        imputed_data3 = imputer3(cc_3, mask3, impu_noise)
        cc_hat3 = cc_3.clone()
        cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3
        reconstruction_loss += l1_loss(imputed_data3, cc_3[:,np.logical_or(missing_col1,missing_col2)].float())

        loss = -critic(cc_hat1).mean()-critic(cc_hat2).mean()-critic(cc_hat3).mean() + weight*reconstruction_loss
        
        imputer1.zero_grad()
        imputer2.zero_grad()
        imputer3.zero_grad()
        loss.backward()
        imputer_optimizer.step()

    scheduler1.step()
    scheduler2.step()

  with torch.no_grad():
    imputer1.eval()
    imputer2.eval()
    imputer3.eval()
    inc_data_pat1 = torch.from_numpy(rawdata_inc1).to(device)
    inc_data_pat2 = torch.from_numpy(rawdata_inc2).to(device)
    inc_data_pat3 = torch.from_numpy(rawdata_inc3).to(device)
    mask1 = torch.from_numpy(missing_col1).to(device)
    mask2 = torch.from_numpy(missing_col2).to(device)
    mask3 = torch.from_numpy(np.logical_and(missing_col1,missing_col2)).to(device)
    for i in range(1):
      impu_noise = torch.empty(sum(np.logical_and(missing_row1,np.logical_not(missing_row2))), p_f+1, device=device)
      imputed_data = imputer1(inc_data_pat1.float(), mask1.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc1[:,missing_col1] = imputed_data.cpu().numpy()
      rawdata[np.logical_and(missing_row1,np.logical_not(missing_row2))] = rawdata_inc1.copy()
      
      impu_noise = torch.empty(sum(np.logical_and(missing_row2,np.logical_not(missing_row1))), p_f+1, device=device)
      imputed_data = imputer2(inc_data_pat2.float(), mask2.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc2[:,missing_col2] = imputed_data.cpu().numpy()
      rawdata[np.logical_and(missing_row2,np.logical_not(missing_row1))] = rawdata_inc2.copy()

      impu_noise = torch.empty(sum(np.logical_and(missing_row1,missing_row2)), p_f+1, device=device)
      imputed_data = imputer3(inc_data_pat3.float(), mask3.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data.cpu().numpy()
      rawdata[np.logical_and(missing_row1, missing_row2)] = rawdata_inc3.copy()
  
  alist = []
  for j in range(2+MI):
    rawdata_pat1 = rawdata[np.logical_and(missing_row1,missing_row2==False),:].copy()
    rawdata_pat2 = rawdata[np.logical_and(missing_row1==False,missing_row2),:].copy()
    rawdata_pat3 = rawdata[np.logical_and(missing_row1,missing_row2),:].copy()
    train_pat1 = rawdata[np.logical_or(missing_row1==False,missing_row2),:].copy()
    train_pat1 = Generate_data_model_2(train_pat1, missing_col1)
    train_pat2 = rawdata[np.logical_or(missing_row1,missing_row2==False),:].copy()
    train_pat2 =  Generate_data_model_2(train_pat2, missing_col2)
    train_pat3 = rawdata[np.logical_or(missing_row1==False,missing_row2==False),:].copy()
    train_pat3 =  Generate_data_model_2(train_pat3, np.logical_or(missing_col1,missing_col2))

    train_pat1_loader1 = DataLoader(train_pat1, batch_size=Batch_size, shuffle=True)
    train_pat1_loader2 = DataLoader(train_pat1, batch_size=Batch_size, shuffle=True)
    train_pat2_loader1 = DataLoader(train_pat2, batch_size=Batch_size, shuffle=True)
    train_pat2_loader2 = DataLoader(train_pat2, batch_size=Batch_size, shuffle=True)
    train_pat3_loader1 = DataLoader(train_pat3, batch_size=Batch_size, shuffle=True)
    train_pat3_loader2 = DataLoader(train_pat3, batch_size=Batch_size, shuffle=True)

    imputer1 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(missing_col1)).to(device) 
    critic1 = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)  
    imputer2 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(missing_col2)).to(device) 
    critic2 = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)
    imputer3 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(np.logical_or(missing_col1,missing_col2))).to(device) 
    critic3 = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)
    
    imputer_params = list(imputer1.parameters()) + list(imputer2.parameters()) + list(imputer3.parameters())
    critic_params = list(critic1.parameters()) + list(critic2.parameters()) + list(critic3.parameters())
    imputer_optimizer = optim.Adam(imputer_params, lr=ilr, betas=(.5, .9))
    critic_optimizer = optim.Adam(critic_params, lr=clr, betas=(.5, .9))
    scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
    scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)

    critic_updates = 0
    for epoch in range(1,1+epochs):
      for cc_1, mask1 in train_pat1_loader1:      
        cc_1 = cc_1.to(device).float()                           
        batch_size1 = cc_1.shape[0]
        impu_noise1 = torch.empty(batch_size1, p_f+1, device=device)
      
        cc_hat1, mask1 = next(iter(train_pat1_loader2))
        cc_hat1 = cc_hat1[0:batch_size1].to(device).float()
        mask1 = mask1[0:batch_size1].to(device).float()
        imputed_data1 = imputer1(cc_hat1, mask1, impu_noise1.normal_(mean=0,std=1))
        cc_hat1[:,missing_col1] = imputed_data1
        
        cc_2, mask2 = next(iter(train_pat2_loader1))
        batch_size2 = cc_2.shape[0]
        impu_noise2 = torch.empty(batch_size2, p_f+1, device=device)
        cc_2 = cc_2[0:batch_size2].to(device).float()
        cc_hat2, mask2 = next(iter(train_pat2_loader2))
        cc_hat2 = cc_hat2[0:batch_size2].to(device).float()
        mask2 = mask2[0:batch_size2].to(device).float()
        imputed_data2 = imputer2(cc_hat2, mask2, impu_noise2.normal_(mean=0,std=1))
        cc_hat2[:,missing_col2] = imputed_data2
        
        cc_3, mask3 = next(iter(train_pat3_loader1))
        batch_size3 = cc_3.shape[0]
        impu_noise3 = torch.empty(batch_size3, p_f+1, device=device)
        cc_3 = cc_3[0:batch_size3].to(device).float()
        cc_hat3, mask3 = next(iter(train_pat3_loader2))
        cc_hat3 = cc_hat3[0:batch_size3].to(device).float()
        mask3 = mask3[0:batch_size3].to(device).float()
        imputed_data3 = imputer3(cc_hat3, mask3, impu_noise3.normal_(mean=0,std=1))
        cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3

        update_critic = CriticUpdater_model_3(critic1, critic2, critic3, critic_optimizer, batch_size1, batch_size2, batch_size3, device=device, gp_lambda=gp_lambda)
        update_critic(cc_1, cc_2, cc_3, cc_hat1, cc_hat2, cc_hat3)
        critic_updates += 1

        if critic_updates == n_critic:
          critic_updates = 0
          reconstruction_loss = 0

          imputed_data1 = imputer1(cc_hat1, mask1, impu_noise1.normal_(mean=0,std=1))
          cc_hat1[:,missing_col1] = imputed_data1
          reconstruction_loss += l1_loss(imputed_data1, cc_1[:,missing_col1].float())
          
          imputed_data2 = imputer2(cc_hat2, mask2, impu_noise2.normal_(mean=0,std=1))
          cc_hat2[:,missing_col2] = imputed_data2
          reconstruction_loss += l1_loss(imputed_data2, cc_2[:,missing_col2].float())
          
          imputed_data3 = imputer3(cc_hat3, mask3, impu_noise3.normal_(mean=0,std=1))
          cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3
          reconstruction_loss += l1_loss(imputed_data3, cc_3[:,np.logical_or(missing_col1,missing_col2)].float())

          loss = -critic1(cc_hat1).mean()-critic2(cc_hat2).mean()-critic3(cc_hat3).mean() + weight*reconstruction_loss
          imputer_optimizer.zero_grad()
          loss.backward()
          imputer_optimizer.step()
      scheduler1.step()
      scheduler2.step()
    
    with torch.no_grad():
      imputer1.eval()
      imputer2.eval()
      imputer3.eval()
      inc_data_pat1 = torch.from_numpy(rawdata_pat1).to(device)
      inc_data_pat2 = torch.from_numpy(rawdata_pat2).to(device)
      inc_data_pat3 = torch.from_numpy(rawdata_pat3).to(device)
      mask1 = torch.from_numpy(missing_col1).to(device)
      mask2 = torch.from_numpy(missing_col2).to(device)
      mask3 = torch.from_numpy(np.logical_or(missing_col1,missing_col2)).to(device)
      for i in range(1):
        torch.manual_seed(j)
        impu_noise = torch.empty(sum(np.logical_and(missing_row1,missing_row2==False)), p_f+1, device=device)
        imputed_data = imputer1(inc_data_pat1.float(), mask1.float(), impu_noise.normal_(mean=0,std=1)) 
        rawdata_pat1[:,missing_col1] = imputed_data.cpu().numpy()
        rawdata[np.logical_and(missing_row1,missing_row2==False)] = rawdata_pat1.copy()
        
        impu_noise = torch.empty(sum(np.logical_and(missing_row1==False,missing_row2)), p_f+1, device=device)
        imputed_data = imputer2(inc_data_pat2.float(), mask2.float(), impu_noise.normal_(mean=0,std=1)) 
        rawdata_pat2[:,missing_col2] = imputed_data.cpu().numpy()
        rawdata[np.logical_and(missing_row1==False,missing_row2)] = rawdata_pat2.copy()
        
        impu_noise = torch.empty(sum(np.logical_and(missing_row1,missing_row2)), p_f+1, device=device)
        imputed_data = imputer3(inc_data_pat3.float(), mask3.float(), impu_noise.normal_(mean=0,std=1)) 
        rawdata_pat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data.cpu().numpy()
        rawdata[np.logical_and(missing_row1,missing_row2)] = rawdata_pat3.copy()

    if j > 1:
      alist.append(rawdata.copy())
  return alist

## test

In [ ]:
Model_2_SD=[]
Model_2_SE_list=[]
Model_2_SE=0
Model_2_MSE=0
Model_2_bias=0
Model_2_coverage=0
Model_2_time=0

a0=1
a1=-2
a2=3
a3=0
a4=2
a5=-2
rho=0.95
sigma1=0.1
sigma2=0.5
Monte_Carlo =100
MI=10
batch_size=256
ilr=1e-3 
clr=1e-3 
gamma=0.9
weight=0.1 
gp_lambda=10 
epochs=300

for iteration in range(1,1+Monte_Carlo):
  obs,mis1,mis2,missing_row1,missing_row2,missing_col1,missing_col2,y=data_G_linear(n=n,p=p,true_beta=true_beta,seed=iteration,a0=a0,a1=a1,a2=a2,a3=a3,a4=a4,a5=a5,rho=rho,sigma1=sigma1,sigma2=sigma2)
  truth1=mis1[missing_row1==True,:].copy()
  truth2=mis2[missing_row2==True,:].copy()

  mis1[missing_row1==True,:]=0
  mis2[missing_row2==True,:]=0
  rawdata = np.concatenate((obs,mis1,mis2,y),axis=1).copy()
  
  s=time.time()
  # alist=[]
  # for i in range(MI):
  #   data = mlp_MIGAN2_version2(rawdata,missing_row1,missing_row2,missing_col1,missing_col2,n=n,p_f=p,Batch_size=batch_size,ilr=ilr,clr=clr,gamma=gamma,weight=weight,gp_lambda=gp_lambda,epochs=epochs,MI_seed=MI)[0]
  #   alist.append(data)
  
  alist = mlp_MIGAN2_iterative2(rawdata,missing_row1,missing_row2,missing_col1,missing_col2,n=n,p_f=p,Batch_size=batch_size,ilr=ilr,clr=clr,gamma=gamma,weight=weight,gp_lambda=gp_lambda,epochs=epochs,MI=MI)

  mse,b,c,se=data_to_stats(alist,true_beta,truth1,truth2,missing_row1,missing_row2,missing_col1,missing_col2)
  bias,cover=Performance(b,c,true_beta,alpha)
  print('Model_2 cost ',time.time()-s,' sec \n','MSE:', mse,'; Bias: ',bias,'; Coverage: ',cover)
  Model_2_SE_list.append(se)
  Model_2_SE=(Model_2_SE*(iteration-1)+se)/iteration
  Model_2_SD.append(b[0])
  Model_2_MSE=(Model_2_MSE*(iteration-1)+mse)/iteration
  Model_2_bias=(Model_2_bias*(iteration-1)+bias)/iteration
  Model_2_coverage=(Model_2_coverage*(iteration-1)+cover)/iteration
  Model_2_time = (Model_2_time*(iteration-1)+time.time()-s)/iteration

  if iteration%5==0:
    print('iteration: ',iteration, 'Model_2_MSE: ',Model_2_MSE,'Model_2_bias: ',Model_2_bias,'Model_2_coverage: ',Model_2_coverage,'Model_2_SE: ',Model_2_SE)
    
    Model_2_StD=np.std(Model_2_SD, dtype=np.float64)
    print('Model_2_StD: ',Model_2_StD)
    # Model_2_SE_median=statistics.median(Model_2_SE_list)
    print('Model_2 average cost:', Model_2_time/MI)

    print('n:', n, ' p: ',p, ' Monte Carlo:',Monte_Carlo, 'MI:',MI,'batch_size:',batch_size,'ilr:',ilr,'clr:',clr,'gamma:',gamma,'weight:',weight,'gp_lambda:',gp_lambda,'epochs:',epochs)
    print('a0=',a0,' a1=', a1,' a2=',a2,' a3=',a3,' a4=',a4,' a5=',a5,' rho=',rho,' sigma1=',sigma1,' sigma2=',sigma2)

# ***MIGAN1***

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import grad
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data import WeightedRandomSampler
from torchvision import datasets, transforms
import torch.nn.functional as F

In [ ]:
class Generate_data_model(Dataset):   
  def __init__(self, array, transform=None):  
    self.array = array
    self.transform = transform

  def __len__(self):
    return len(self.array)

  def __getitem__(self, index):
    return self.array[index]

class Generate_data_model_2(Dataset):   
  def __init__(self, array, mask, transform=None):  
    self.array = array
    self.mask = mask             
    self.transform = transform

  def __len__(self):
    return len(self.array)

  def __getitem__(self, index):
    return self.array[index], self.mask

class Imputer_model_2(nn.Module):
  def __init__(self, n1=251, n2=200, n3=100, n4=100):
    super().__init__()
    self.fc = nn.Sequential(
        nn.Linear(n1, n2),
        # nn.ReLU(),
        nn.Tanh(),
        nn.Linear(n2, n3),
        # nn.ReLU(),
        nn.Tanh(),
        nn.Linear(n3, n4),
    )
  def forward(self, data, mask, noise):
    net = data * (1 - mask) + noise * mask
    net = net.view(data.shape[0], -1)
    net = self.fc(net.float())
    net = net.view(data.shape[0], -1)
    return net

class Critic_model_2(nn.Module):
  def __init__(self, n1=251, n2=200, n3=50, n4=1):  #n1=251, n2=100, n3=20, n4=1
    super().__init__()
    self.fc = nn.Sequential(
        nn.Linear(n1, n2),
        nn.ReLU(),   
        # nn.Tanh(),
        nn.Linear(n2, n3),
        nn.ReLU(),   
        # nn.Tanh(),
        nn.Linear(n3, n4),
    )
  def forward(self, x):
    out = self.fc(x)
    return out

class CriticUpdater_model:
  def __init__(self, critic, critic_optimizer, batch_size=8, gp_lambda=10, device='cuda:0'):
    self.critic = critic
    self.critic_optimizer = critic_optimizer
    self.gp_lambda = gp_lambda
    # Interpolation coefficient
    self.eps = torch.empty(batch_size, 1,device=device)
    # For computing the gradient penalty
    self.ones = torch.ones(batch_size, 1).to(device)

  def __call__(self, real, fake, device='cuda:0'):
    real = real.detach()
    fake = fake.detach()
    self.critic.zero_grad()
    self.eps.uniform_(0, 1)
    interp = (self.eps * real + (1 - self.eps) * fake).to(device).requires_grad_()
    grad_d = grad(outputs=self.critic(interp.float()), inputs=interp, grad_outputs=self.ones,create_graph=True)[0]  #grad_outputs=self.ones,create_graph=True
    grad_d = grad_d.view(real.shape[0], -1)
    grad_penalty = ((grad_d.norm(dim=1) - 1)**2).mean() * self.gp_lambda
    w_dist = self.critic(fake.float()).mean() - self.critic(real.float()).mean()
    loss = w_dist + grad_penalty
    loss.backward()
    self.critic_optimizer.step()

class CriticUpdater_model_2:
  def __init__(self, critic, critic_optimizer, batch_size=8, gp_lambda=10, device='cuda:0'):
    self.critic = critic
    self.critic_optimizer = critic_optimizer
    self.gp_lambda = gp_lambda
    # Interpolation coefficient
    self.eps = torch.empty(batch_size, 1,device=device)
    # For computing the gradient penalty
    self.ones = torch.ones(batch_size, 1).to(device)

  def __call__(self, real, fake1, fake2, fake3, device='cuda:0'):
    real = real.detach()
    fake1 = fake1.detach()
    fake2 = fake2.detach()
    fake3 = fake3.detach()
    self.critic.zero_grad()
    self.eps.uniform_(0, 1)
    interp1 = (self.eps * real + (1 - self.eps) * fake1).to(device).requires_grad_()
    self.eps.uniform_(0, 1)
    interp2 = (self.eps * real + (1 - self.eps) * fake2).to(device).requires_grad_()
    self.eps.uniform_(0, 1)
    interp3 = (self.eps * real + (1 - self.eps) * fake3).to(device).requires_grad_()
    grad_d1 = grad(outputs=self.critic(interp1.float()), inputs=interp1, grad_outputs=self.ones,create_graph=True)[0].view(real.shape[0], -1)  #grad_outputs=self.ones,create_graph=True
    grad_d2 = grad(outputs=self.critic(interp2.float()), inputs=interp2, grad_outputs=self.ones,create_graph=True)[0].view(real.shape[0], -1)
    grad_d3 = grad(outputs=self.critic(interp3.float()), inputs=interp3, grad_outputs=self.ones,create_graph=True)[0].view(real.shape[0], -1)
    grad_penalty1 = ((grad_d1.norm(dim=1) - 1)**2).mean() * self.gp_lambda
    grad_penalty2 = ((grad_d2.norm(dim=1) - 1)**2).mean() * self.gp_lambda
    grad_penalty3 = ((grad_d3.norm(dim=1) - 1)**2).mean() * self.gp_lambda
    w_dist1 = self.critic(fake1.float()).mean() - self.critic(real.float()).mean()
    w_dist2 = self.critic(fake2.float()).mean() - self.critic(real.float()).mean()
    w_dist3 = self.critic(fake3.float()).mean() - self.critic(real.float()).mean()
    loss = w_dist1 + w_dist2 + w_dist3 + grad_penalty1 + grad_penalty2 + grad_penalty3 
    loss.backward()
    self.critic_optimizer.step()

def mlp_MIGAN1(missing_data, missing_row1,missing_row2,missing_col1,missing_col2, n=200, p_f=250, device='cuda:0', Batch_size=16, ilr=1e-3, clr=1e-3, gamma=0.2, weight=0.1, gp_lambda=10, epochs=300, MI_seed=1):
  rawdata = missing_data.copy()
  rawdata_inc1 = rawdata[np.logical_and(missing_row1,np.logical_not(missing_row2)), :].copy()
  rawdata_inc2 = rawdata[np.logical_and(missing_row2,np.logical_not(missing_row1)), :].copy()
  rawdata_inc3 = rawdata[np.logical_and(missing_row1,missing_row2), :].copy()
  cc = torch.from_numpy(rawdata.copy()[np.logical_or(missing_row1,missing_row2)==0, :])

  cc_data = Generate_data_model(cc)
  cc_data_pat1 =  Generate_data_model_2(cc, missing_col1)
  cc_data_pat2 =  Generate_data_model_2(cc, missing_col2)
  cc_data_pat3 =  Generate_data_model_2(cc, np.logical_or(missing_col1,missing_col2))

  cc_data_loader = DataLoader(cc_data, batch_size=Batch_size, shuffle=True)
  cc_data_loader1 = DataLoader(cc_data_pat1, batch_size=Batch_size, shuffle=True)
  cc_data_loader2 = DataLoader(cc_data_pat2, batch_size=Batch_size, shuffle=True)
  cc_data_loader3 = DataLoader(cc_data_pat3, batch_size=Batch_size, shuffle=True)
  
  imputer1 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=p_f+1).to(device)
  imputer2 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=p_f+1).to(device)
  imputer3 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=p_f+1).to(device)  

  critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)    
  imputer_params = list(imputer1.parameters()) + list(imputer2.parameters()) + list(imputer3.parameters())
  imputer_optimizer = optim.Adam(imputer_params, lr=ilr, betas=(.5, .9))
  critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
  scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
  scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
  mse_loss = torch.nn.MSELoss(reduction='mean')
  l1_loss = nn.L1Loss(reduction='mean')

  critic_updates = 0
  n_critic = 5

  for epoch in range(1,1+epochs):
    for cc in cc_data_loader:      
      cc = cc.to(device).float()                           
      batch_size = cc.shape[0]
      impu_noise = torch.empty(batch_size, p_f+1, device=device)
     
      cc_1, mask1 = next(iter(cc_data_loader1))
      cc_1 = cc_1[0:batch_size].to(device).float()
      mask1 = mask1[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data1 = imputer1(cc_1, mask1, impu_noise)
      cc_hat1 = cc_1.clone()
      cc_hat1[:,missing_col1] = imputed_data1[:,missing_col1]

      cc_2, mask2 = next(iter(cc_data_loader2))
      cc_2 = cc_2[0:batch_size].to(device).float()
      mask2 = mask2[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data2 = imputer2(cc_2, mask2, impu_noise)
      cc_hat2 = cc_2.clone()
      cc_hat2[:,missing_col2] = imputed_data2[:,missing_col2]

      cc_3, mask3 = next(iter(cc_data_loader3))
      cc_3 = cc_3[0:batch_size].to(device).float()
      mask3 = mask3[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data3 = imputer3(cc_3, mask3, impu_noise)
      cc_hat3 = cc_3.clone()
      cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3[:,np.logical_or(missing_col1,missing_col2)]
      
      update_critic = CriticUpdater_model_2(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
      update_critic(cc, cc_hat1, cc_hat2, cc_hat3)
      critic_updates += 1

      if critic_updates == n_critic:
        critic_updates = 0
        reconstruction_loss = 0

        impu_noise.normal_(mean=0,std=1)
        imputed_data1 = imputer1(cc_1, mask1, impu_noise)
        cc_hat1 = cc_1.clone()
        cc_hat1[:,missing_col1] = imputed_data1[:,missing_col1]
        reconstruction_loss += l1_loss(imputed_data1, cc_1)
        
        impu_noise.normal_(mean=0,std=1)
        imputed_data2 = imputer2(cc_2, mask2, impu_noise)
        cc_hat2 = cc_2.clone()
        cc_hat2[:,missing_col2] = imputed_data2[:,missing_col2]
        reconstruction_loss += l1_loss(imputed_data2, cc_2)

        impu_noise.normal_(mean=0,std=1)
        imputed_data3 = imputer3(cc_3, mask3, impu_noise)
        cc_hat3 = cc_3.clone()
        cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3[:,np.logical_or(missing_col1,missing_col2)]
        reconstruction_loss += l1_loss(imputed_data3, cc_3)

        loss = -critic(cc_hat1).mean()-critic(cc_hat2).mean()-critic(cc_hat3).mean() + weight*reconstruction_loss
        
        imputer1.zero_grad()
        imputer2.zero_grad()
        imputer3.zero_grad()
        loss.backward()
        imputer_optimizer.step()

    scheduler1.step()
    scheduler2.step()

  alist = []
  with torch.no_grad():
    imputer1.eval()
    imputer2.eval()
    imputer3.eval()
    inc_data_pat1 = torch.from_numpy(rawdata_inc1).to(device)
    inc_data_pat2 = torch.from_numpy(rawdata_inc2).to(device)
    inc_data_pat3 = torch.from_numpy(rawdata_inc3).to(device)
    mask1 = torch.from_numpy(missing_col1).to(device)
    mask2 = torch.from_numpy(missing_col2).to(device)
    mask3 = torch.from_numpy(np.logical_and(missing_col1,missing_col2)).to(device)
    for i in range(1):
      torch.manual_seed(MI_seed)
      impu_noise = torch.empty(sum(np.logical_and(missing_row1,np.logical_not(missing_row2))), p_f+1, device=device)
      imputed_data = imputer1(inc_data_pat1.float(), mask1.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc1[:,missing_col1] = imputed_data.cpu().numpy()[:,missing_col1]
      # print(sum(np.isnan(imputed_data.cpu().numpy())))
      rawdata[np.logical_and(missing_row1,np.logical_not(missing_row2))] = rawdata_inc1.copy()
      
      impu_noise = torch.empty(sum(np.logical_and(missing_row2,np.logical_not(missing_row1))), p_f+1, device=device)
      imputed_data = imputer2(inc_data_pat2.float(), mask2.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc2[:,missing_col2] = imputed_data.cpu().numpy()[:,missing_col2]
      # print(sum(np.isnan(imputed_data.cpu().numpy())))
      rawdata[np.logical_and(missing_row2,np.logical_not(missing_row1))] = rawdata_inc2.copy()

      impu_noise = torch.empty(sum(np.logical_and(missing_row1,missing_row2)), p_f+1, device=device)
      imputed_data = imputer3(inc_data_pat3.float(), mask3.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data.cpu().numpy()[:,np.logical_or(missing_col1,missing_col2)]
      # print(sum(np.isnan(imputed_data.cpu().numpy())))
      rawdata[np.logical_and(missing_row1, missing_row2)] = rawdata_inc3.copy()

      alist.append(rawdata.copy())
  return alist

In [ ]:
def mlp_MIGAN1_version2(missing_data, missing_row1,missing_row2,missing_col1,missing_col2, n=200, p_f=250, device='cuda:0', Batch_size=16, ilr=1e-3, clr=1e-3, gamma=0.2, weight=0.1, gp_lambda=10, epochs=300, MI_seed=10):
  rawdata = missing_data.copy()
  rawdata_inc1 = rawdata[np.logical_and(missing_row1,np.logical_not(missing_row2)), :].copy()
  rawdata_inc2 = rawdata[np.logical_and(missing_row2,np.logical_not(missing_row1)), :].copy()
  rawdata_inc3 = rawdata[np.logical_and(missing_row1,missing_row2), :].copy()
  cc = torch.from_numpy(rawdata.copy()[np.logical_or(missing_row1,missing_row2)==0, :])

  cc_data = Generate_data_model(cc)
  cc_data_pat1 =  Generate_data_model_2(cc, missing_col1)
  cc_data_pat2 =  Generate_data_model_2(cc, missing_col2)
  cc_data_pat3 =  Generate_data_model_2(cc, np.logical_or(missing_col1,missing_col2))

  cc_data_loader = DataLoader(cc_data, batch_size=Batch_size, shuffle=True)
  cc_data_loader1 = DataLoader(cc_data_pat1, batch_size=Batch_size, shuffle=True)
  cc_data_loader2 = DataLoader(cc_data_pat2, batch_size=Batch_size, shuffle=True)
  cc_data_loader3 = DataLoader(cc_data_pat3, batch_size=Batch_size, shuffle=True)
  
  mse_loss = torch.nn.MSELoss(reduction='mean')
  l1_loss = nn.L1Loss(reduction='mean')
  critic_updates = 0
  n_critic = 5
  
  imputer1 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=p_f+1).to(device) 
  critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)    
  imputer_optimizer = optim.Adam(imputer1.parameters(), lr=ilr, betas=(.5, .9))
  critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
  scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
  scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
  
  critic_updates = 0
  for epoch in range(1,1+epochs):
    for cc in cc_data_loader:      
      cc = cc.to(device).float()                           
      batch_size = cc.shape[0]
      impu_noise = torch.empty(batch_size, p_f+1, device=device)
    
      cc_1, mask1 = next(iter(cc_data_loader1))
      cc_1 = cc_1[0:batch_size].to(device).float()
      mask1 = mask1[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data1 = imputer1(cc_1, mask1, impu_noise)
      cc_hat1 = cc_1.clone()
      cc_hat1[:,missing_col1] = imputed_data1[:,missing_col1]
      update_critic = CriticUpdater_model(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
      update_critic(cc, cc_hat1)
      critic_updates += 1

      if critic_updates == n_critic:
        critic_updates = 0
        reconstruction_loss = 0

        impu_noise.normal_(mean=0,std=1)
        imputed_data1 = imputer1(cc_1, mask1, impu_noise)
        cc_hat1 = cc_1.clone()
        cc_hat1[:,missing_col1] = imputed_data1[:,missing_col1]
        reconstruction_loss += l1_loss(imputed_data1, cc_1.float())
        loss = -critic(cc_hat1).mean() + weight*reconstruction_loss
        imputer1.zero_grad()
        loss.backward()
        imputer_optimizer.step()
    scheduler1.step()
    scheduler2.step()
  
  imputer2 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=p_f+1).to(device) 
  critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)
  imputer_optimizer = optim.Adam(imputer2.parameters(), lr=ilr, betas=(.5, .9))
  critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
  scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
  scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
  critic_updates = 0

  for epoch in range(1,1+epochs):
    for cc in cc_data_loader:      
      cc = cc.to(device).float()                           
      batch_size = cc.shape[0]
      impu_noise = torch.empty(batch_size, p_f+1, device=device)
    
      cc_2, mask2 = next(iter(cc_data_loader2))
      cc_2 = cc_2[0:batch_size].to(device).float()
      mask2 = mask2[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data2 = imputer2(cc_2, mask2, impu_noise)
      cc_hat2 = cc_2.clone()
      cc_hat2[:,missing_col2] = imputed_data2[:,missing_col2]

      update_critic = CriticUpdater_model(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
      update_critic(cc, cc_hat2)
      critic_updates += 1

      if critic_updates == n_critic:
        critic_updates = 0
        reconstruction_loss = 0
        
        impu_noise.normal_(mean=0,std=1)
        imputed_data2 = imputer2(cc_2, mask2, impu_noise)
        cc_hat2 = cc_2.clone()
        cc_hat2[:,missing_col2] = imputed_data2[:,missing_col2]
        reconstruction_loss += l1_loss(imputed_data2, cc_2.float())
        loss = -critic(cc_hat2).mean() + weight*reconstruction_loss
        imputer2.zero_grad()
        loss.backward()
        imputer_optimizer.step()
    scheduler1.step()
    scheduler2.step()


  imputer3 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=p_f+1).to(device) 
  critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)
  imputer_optimizer = optim.Adam(imputer3.parameters(), lr=ilr, betas=(.5, .9))
  critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
  scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
  scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
  critic_updates = 0

  for epoch in range(1,1+epochs):
    for cc in cc_data_loader:      
      cc = cc.to(device).float()                           
      batch_size = cc.shape[0]
      impu_noise = torch.empty(batch_size, p_f+1, device=device)
    
      cc_3, mask3 = next(iter(cc_data_loader3))
      cc_3 = cc_3[0:batch_size].to(device).float()
      mask3 = mask3[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data3 = imputer3(cc_3, mask3, impu_noise)
      cc_hat3 = cc_3.clone()
      cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3[:,np.logical_or(missing_col1,missing_col2)]

      update_critic = CriticUpdater_model(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
      update_critic(cc, cc_hat3)
      critic_updates += 1

      if critic_updates == n_critic:
        critic_updates = 0
        reconstruction_loss = 0
        
        impu_noise.normal_(mean=0,std=1)
        imputed_data3 = imputer3(cc_3, mask3, impu_noise)
        cc_hat3 = cc_3.clone()
        cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3[:,np.logical_or(missing_col1,missing_col2)]
        reconstruction_loss += l1_loss(imputed_data3, cc_3.float())
        loss = -critic(cc_hat3).mean() + weight*reconstruction_loss
        imputer3.zero_grad()
        loss.backward()
        imputer_optimizer.step()
    scheduler1.step()
    scheduler2.step()
  
  alist=[]
  with torch.no_grad():
    imputer1.eval()
    imputer2.eval()
    imputer3.eval()
    inc_data_pat1 = torch.from_numpy(rawdata_inc1).to(device)
    inc_data_pat2 = torch.from_numpy(rawdata_inc2).to(device)
    inc_data_pat3 = torch.from_numpy(rawdata_inc3).to(device)
    mask1 = torch.from_numpy(missing_col1).to(device)
    mask2 = torch.from_numpy(missing_col2).to(device)
    mask3 = torch.from_numpy(np.logical_and(missing_col1,missing_col2)).to(device)
    for i in range(1):
      torch.manual_seed(MI_seed)
      impu_noise = torch.empty(sum(np.logical_and(missing_row1,np.logical_not(missing_row2))), p_f+1, device=device)
      imputed_data = imputer1(inc_data_pat1.float(), mask1.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc1[:,missing_col1] = imputed_data.cpu().numpy()[:,missing_col1]
      rawdata[np.logical_and(missing_row1,np.logical_not(missing_row2))] = rawdata_inc1.copy()
      
      impu_noise = torch.empty(sum(np.logical_and(missing_row2,np.logical_not(missing_row1))), p_f+1, device=device)
      imputed_data = imputer2(inc_data_pat2.float(), mask2.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc2[:,missing_col2] = imputed_data.cpu().numpy()[:,missing_col2]
      rawdata[np.logical_and(missing_row2,np.logical_not(missing_row1))] = rawdata_inc2.copy()

      impu_noise = torch.empty(sum(np.logical_and(missing_row1,missing_row2)), p_f+1, device=device)
      imputed_data = imputer3(inc_data_pat3.float(), mask3.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data.cpu().numpy()[:,np.logical_or(missing_col1,missing_col2)]
      rawdata[np.logical_and(missing_row1, missing_row2)] = rawdata_inc3.copy()

      alist.append(rawdata.copy())
  return alist

## iterative approach, column pattern 

In [ ]:
def mlp_MIGAN1_iterative(missing_data, missing_row1,missing_row2,missing_col1,missing_col2, n=200, p_f=250, device='cuda:0', Batch_size=16, ilr=1e-3, clr=1e-3, gamma=0.2, weight=0.1, gp_lambda=10, epochs=300, MI=10):
  rawdata = missing_data.copy()
  rawdata_inc1 = rawdata[np.logical_and(missing_row1,np.logical_not(missing_row2)), :].copy()
  rawdata_inc2 = rawdata[np.logical_and(missing_row2,np.logical_not(missing_row1)), :].copy()
  rawdata_inc3 = rawdata[np.logical_and(missing_row1,missing_row2), :].copy()
  cc = torch.from_numpy(rawdata.copy()[np.logical_or(missing_row1,missing_row2)==0, :])

  cc_data = Generate_data_model(cc)
  cc_data_pat1 =  Generate_data_model_2(cc, missing_col1)
  cc_data_pat2 =  Generate_data_model_2(cc, missing_col2)
  cc_data_pat3 =  Generate_data_model_2(cc, np.logical_or(missing_col1,missing_col2))

  cc_data_loader = DataLoader(cc_data, batch_size=Batch_size, shuffle=True)
  cc_data_loader1 = DataLoader(cc_data_pat1, batch_size=Batch_size, shuffle=True)
  cc_data_loader2 = DataLoader(cc_data_pat2, batch_size=Batch_size, shuffle=True)
  cc_data_loader3 = DataLoader(cc_data_pat3, batch_size=Batch_size, shuffle=True)
  
  imputer1 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=p_f+1).to(device)
  imputer2 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=p_f+1).to(device)
  imputer3 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=p_f+1).to(device)  

  critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)    
  imputer_params = list(imputer1.parameters()) + list(imputer2.parameters()) + list(imputer3.parameters())
  imputer_optimizer = optim.Adam(imputer_params, lr=ilr, betas=(.5, .9))
  critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
  scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
  scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
  mse_loss = torch.nn.MSELoss(reduction='mean')
  l1_loss = nn.L1Loss(reduction='mean')

  critic_updates = 0
  n_critic = 5

  for epoch in range(1,1+epochs):
    for cc in cc_data_loader:      
      cc = cc.to(device).float()                           
      batch_size = cc.shape[0]
      impu_noise = torch.empty(batch_size, p_f+1, device=device)
     
      cc_1, mask1 = next(iter(cc_data_loader1))
      cc_1 = cc_1[0:batch_size].to(device).float()
      mask1 = mask1[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data1 = imputer1(cc_1, mask1, impu_noise)
      cc_hat1 = cc_1.clone()
      cc_hat1[:,missing_col1] = imputed_data1[:,missing_col1]

      cc_2, mask2 = next(iter(cc_data_loader2))
      cc_2 = cc_2[0:batch_size].to(device).float()
      mask2 = mask2[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data2 = imputer2(cc_2, mask2, impu_noise)
      cc_hat2 = cc_2.clone()
      cc_hat2[:,missing_col2] = imputed_data2[:,missing_col2]

      cc_3, mask3 = next(iter(cc_data_loader3))
      cc_3 = cc_3[0:batch_size].to(device).float()
      mask3 = mask3[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data3 = imputer3(cc_3, mask3, impu_noise)
      cc_hat3 = cc_3.clone()
      cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3[:,np.logical_or(missing_col1,missing_col2)]
      
      update_critic = CriticUpdater_model_2(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
      update_critic(cc, cc_hat1, cc_hat2, cc_hat3)
      critic_updates += 1

      if critic_updates == n_critic:
        critic_updates = 0
        reconstruction_loss = 0

        impu_noise.normal_(mean=0,std=1)
        imputed_data1 = imputer1(cc_1, mask1, impu_noise)
        cc_hat1 = cc_1.clone()
        cc_hat1[:,missing_col1] = imputed_data1[:,missing_col1]
        reconstruction_loss += l1_loss(imputed_data1, cc_1)
        
        impu_noise.normal_(mean=0,std=1)
        imputed_data2 = imputer2(cc_2, mask2, impu_noise)
        cc_hat2 = cc_2.clone()
        cc_hat2[:,missing_col2] = imputed_data2[:,missing_col2]
        reconstruction_loss += l1_loss(imputed_data2, cc_2)

        impu_noise.normal_(mean=0,std=1)
        imputed_data3 = imputer3(cc_3, mask3, impu_noise)
        cc_hat3 = cc_3.clone()
        cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3[:,np.logical_or(missing_col1,missing_col2)]
        reconstruction_loss += l1_loss(imputed_data3, cc_3)

        loss = -critic(cc_hat1).mean()-critic(cc_hat2).mean()-critic(cc_hat3).mean() + weight*reconstruction_loss
        
        imputer1.zero_grad()
        imputer2.zero_grad()
        imputer3.zero_grad()
        loss.backward()
        imputer_optimizer.step()

    scheduler1.step()
    scheduler2.step()

  with torch.no_grad():
    imputer1.eval()
    imputer2.eval()
    imputer3.eval()
    inc_data_pat1 = torch.from_numpy(rawdata_inc1).to(device)
    inc_data_pat2 = torch.from_numpy(rawdata_inc2).to(device)
    inc_data_pat3 = torch.from_numpy(rawdata_inc3).to(device)
    mask1 = torch.from_numpy(missing_col1).to(device)
    mask2 = torch.from_numpy(missing_col2).to(device)
    mask3 = torch.from_numpy(np.logical_and(missing_col1,missing_col2)).to(device)
    for i in range(1):
      impu_noise = torch.empty(sum(np.logical_and(missing_row1,np.logical_not(missing_row2))), p_f+1, device=device)
      imputed_data = imputer1(inc_data_pat1.float(), mask1.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc1[:,missing_col1] = imputed_data.cpu().numpy()[:,missing_col1]
      rawdata[np.logical_and(missing_row1,np.logical_not(missing_row2))] = rawdata_inc1.copy()
      
      impu_noise = torch.empty(sum(np.logical_and(missing_row2,np.logical_not(missing_row1))), p_f+1, device=device)
      imputed_data = imputer2(inc_data_pat2.float(), mask2.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc2[:,missing_col2] = imputed_data.cpu().numpy()[:,missing_col2]
      rawdata[np.logical_and(missing_row2,np.logical_not(missing_row1))] = rawdata_inc2.copy()

      impu_noise = torch.empty(sum(np.logical_and(missing_row1,missing_row2)), p_f+1, device=device)
      imputed_data = imputer3(inc_data_pat3.float(), mask3.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data.cpu().numpy()[:,np.logical_or(missing_col1,missing_col2)]
      rawdata[np.logical_and(missing_row1, missing_row2)] = rawdata_inc3.copy()

  alist = []
  for j in range(1+MI):
    rawdata_pat1 = rawdata[missing_row1, :].copy()
    train_pat1 = rawdata[np.logical_not(missing_row1), :].copy()
    train_pat1 =  Generate_data_model_2(train_pat1, missing_col1)
    train_pat1_loader1 = DataLoader(train_pat1, batch_size=Batch_size, shuffle=True)
    train_pat1_loader2 = DataLoader(train_pat1, batch_size=Batch_size, shuffle=True)
    
    imputer1 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=p_f+1).to(device) 
    critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)    
    imputer_optimizer = optim.Adam(imputer1.parameters(), lr=ilr, betas=(.5, .9))
    critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
    scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
    scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
    
    critic_updates = 0
    for epoch in range(1,1+epochs):
      for cc, mask in train_pat1_loader1:      
        cc = cc.to(device).float()                           
        batch_size = cc.shape[0]
        impu_noise = torch.empty(batch_size, p_f+1, device=device)
      
        cc_1, mask1 = next(iter(train_pat1_loader2))
        cc_1 = cc_1[0:batch_size].to(device).float()
        mask1 = mask1[0:batch_size].to(device).float()
        impu_noise.normal_(mean=0,std=1)
        imputed_data1 = imputer1(cc_1, mask1, impu_noise)
        cc_hat1 = cc_1.clone()
        cc_hat1[:,missing_col1] = imputed_data1[:,missing_col1]
        update_critic = CriticUpdater_model(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
        update_critic(cc, cc_hat1)
        critic_updates += 1

        if critic_updates == n_critic:
          critic_updates = 0
          reconstruction_loss = 0

          impu_noise.normal_(mean=0,std=1)
          imputed_data1 = imputer1(cc_1, mask1, impu_noise)
          cc_hat1 = cc_1.clone()
          cc_hat1[:,missing_col1] = imputed_data1[:,missing_col1]
          reconstruction_loss += l1_loss(imputed_data1, cc_1.float())
          loss = -critic(cc_hat1).mean() + weight*reconstruction_loss
          imputer1.zero_grad()
          loss.backward()
          imputer_optimizer.step()
      scheduler1.step()
      scheduler2.step()
    
    with torch.no_grad():
      imputer1.eval()
      inc_data_pat1 = torch.from_numpy(rawdata_pat1).to(device)
      mask1 = torch.from_numpy(missing_col1).to(device)
      for i in range(1):
        torch.manual_seed(j)
        impu_noise = torch.empty(sum(missing_row1), p_f+1, device=device)
        imputed_data = imputer1(inc_data_pat1.float(), mask1.float(), impu_noise.normal_(mean=0,std=1)) 
        rawdata_pat1[:,missing_col1] = imputed_data.cpu().numpy()[:,missing_col1]
        rawdata[missing_row1] = rawdata_pat1.copy()

    rawdata_pat2 = rawdata[missing_row2, :].copy()
    train_pat2 = rawdata[np.logical_not(missing_row2), :].copy()
    train_pat2 =  Generate_data_model_2(train_pat2, missing_col2)
    
    train_pat2_loader1 = DataLoader(train_pat2, batch_size=Batch_size, shuffle=True)
    train_pat2_loader2 = DataLoader(train_pat2, batch_size=Batch_size, shuffle=True)
    imputer2 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=p_f+1).to(device) 
    critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)
    imputer_optimizer = optim.Adam(imputer2.parameters(), lr=ilr, betas=(.5, .9))
    critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
    scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
    scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
    critic_updates = 0

    for epoch in range(1,1+epochs):
      for cc, mask in train_pat2_loader1:      
        cc = cc.to(device).float()                           
        batch_size = cc.shape[0]
        impu_noise = torch.empty(batch_size, p_f+1, device=device)
      
        cc_2, mask2 = next(iter(train_pat2_loader2))
        cc_2 = cc_2[0:batch_size].to(device).float()
        mask2 = mask2[0:batch_size].to(device).float()
        impu_noise.normal_(mean=0,std=1)
        imputed_data2 = imputer2(cc_2, mask2, impu_noise)
        cc_hat2 = cc_2.clone()
        cc_hat2[:,missing_col2] = imputed_data2[:,missing_col2]

        update_critic = CriticUpdater_model(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
        update_critic(cc, cc_hat2)
        critic_updates += 1

        if critic_updates == n_critic:
          critic_updates = 0
          reconstruction_loss = 0
          
          impu_noise.normal_(mean=0,std=1)
          imputed_data2 = imputer2(cc_2, mask2, impu_noise)
          cc_hat2 = cc_2.clone()
          cc_hat2[:,missing_col2] = imputed_data2[:,missing_col2]
          reconstruction_loss += l1_loss(imputed_data2, cc_2.float())
          loss = -critic(cc_hat2).mean() + weight*reconstruction_loss
          imputer2.zero_grad()
          loss.backward()
          imputer_optimizer.step()
      scheduler1.step()
      scheduler2.step()

    with torch.no_grad():
      imputer2.eval()
      inc_data_pat2 = torch.from_numpy(rawdata_pat2).to(device)
      mask2 = torch.from_numpy(missing_col2).to(device)
      for i in range(1):
        torch.manual_seed(j)
        impu_noise = torch.empty(sum(missing_row2), p_f+1, device=device)
        imputed_data = imputer2(inc_data_pat2.float(), mask2.float(), impu_noise.normal_(mean=0,std=1)) 
        rawdata_pat2[:,missing_col2] = imputed_data.cpu().numpy()[:,missing_col2]
        rawdata[missing_row2] = rawdata_pat2.copy()

    if j > 0:
      alist.append(rawdata.copy())
  return alist

## iterative approach, row pattern, 1

In [ ]:
def mlp_MIGAN1_iterative2(missing_data, missing_row1,missing_row2,missing_col1,missing_col2, n=200, p_f=250, device='cuda:0', Batch_size=16, ilr=1e-3, clr=1e-3, gamma=0.2, weight=0.1, gp_lambda=10, epochs=300, MI=10):
  rawdata = missing_data.copy()
  rawdata_inc1 = rawdata[np.logical_and(missing_row1,np.logical_not(missing_row2)), :].copy()
  rawdata_inc2 = rawdata[np.logical_and(missing_row2,np.logical_not(missing_row1)), :].copy()
  rawdata_inc3 = rawdata[np.logical_and(missing_row1,missing_row2), :].copy()
  cc = torch.from_numpy(rawdata.copy()[np.logical_or(missing_row1,missing_row2)==0, :])

  cc_data = Generate_data_model(cc)
  cc_data_pat1 =  Generate_data_model_2(cc, missing_col1)
  cc_data_pat2 =  Generate_data_model_2(cc, missing_col2)
  cc_data_pat3 =  Generate_data_model_2(cc, np.logical_or(missing_col1,missing_col2))

  cc_data_loader = DataLoader(cc_data, batch_size=Batch_size, shuffle=True)
  cc_data_loader1 = DataLoader(cc_data_pat1, batch_size=Batch_size, shuffle=True)
  cc_data_loader2 = DataLoader(cc_data_pat2, batch_size=Batch_size, shuffle=True)
  cc_data_loader3 = DataLoader(cc_data_pat3, batch_size=Batch_size, shuffle=True)
  
  mse_loss = torch.nn.MSELoss(reduction='mean')
  l1_loss = nn.L1Loss(reduction='mean')
  critic_updates = 0
  n_critic = 5
  
  imputer1 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=p_f+1).to(device) 
  critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)    
  imputer_optimizer = optim.Adam(imputer1.parameters(), lr=ilr, betas=(.5, .9))
  critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
  scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
  scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
  
  critic_updates = 0
  for epoch in range(1,1+epochs):
    for cc in cc_data_loader:      
      cc = cc.to(device).float()                           
      batch_size = cc.shape[0]
      impu_noise = torch.empty(batch_size, p_f+1, device=device)
    
      cc_1, mask1 = next(iter(cc_data_loader1))
      cc_1 = cc_1[0:batch_size].to(device).float()
      mask1 = mask1[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data1 = imputer1(cc_1, mask1, impu_noise)
      cc_hat1 = cc_1.clone()
      cc_hat1[:,missing_col1] = imputed_data1[:,missing_col1]
      update_critic = CriticUpdater_model(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
      update_critic(cc, cc_hat1)
      critic_updates += 1

      if critic_updates == n_critic:
        critic_updates = 0
        reconstruction_loss = 0

        impu_noise.normal_(mean=0,std=1)
        imputed_data1 = imputer1(cc_1, mask1, impu_noise)
        cc_hat1 = cc_1.clone()
        cc_hat1[:,missing_col1] = imputed_data1[:,missing_col1]
        reconstruction_loss += l1_loss(imputed_data1, cc_1.float())
        loss = -critic(cc_hat1).mean() + weight*reconstruction_loss
        imputer1.zero_grad()
        loss.backward()
        imputer_optimizer.step()
    scheduler1.step()
    scheduler2.step()
  
  imputer2 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=p_f+1).to(device) 
  critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)
  imputer_optimizer = optim.Adam(imputer2.parameters(), lr=ilr, betas=(.5, .9))
  critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
  scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
  scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
  critic_updates = 0

  for epoch in range(1,1+epochs):
    for cc in cc_data_loader:      
      cc = cc.to(device).float()                           
      batch_size = cc.shape[0]
      impu_noise = torch.empty(batch_size, p_f+1, device=device)
    
      cc_2, mask2 = next(iter(cc_data_loader2))
      cc_2 = cc_2[0:batch_size].to(device).float()
      mask2 = mask2[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data2 = imputer2(cc_2, mask2, impu_noise)
      cc_hat2 = cc_2.clone()
      cc_hat2[:,missing_col2] = imputed_data2[:,missing_col2]

      update_critic = CriticUpdater_model(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
      update_critic(cc, cc_hat2)
      critic_updates += 1

      if critic_updates == n_critic:
        critic_updates = 0
        reconstruction_loss = 0
        
        impu_noise.normal_(mean=0,std=1)
        imputed_data2 = imputer2(cc_2, mask2, impu_noise)
        cc_hat2 = cc_2.clone()
        cc_hat2[:,missing_col2] = imputed_data2[:,missing_col2]
        reconstruction_loss += l1_loss(imputed_data2, cc_2.float())
        loss = -critic(cc_hat2).mean() + weight*reconstruction_loss
        imputer2.zero_grad()
        loss.backward()
        imputer_optimizer.step()
    scheduler1.step()
    scheduler2.step()

  imputer3 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=p_f+1).to(device) 
  critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)
  imputer_optimizer = optim.Adam(imputer3.parameters(), lr=ilr, betas=(.5, .9))
  critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
  scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
  scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
  critic_updates = 0

  for epoch in range(1,1+epochs):
    for cc in cc_data_loader:      
      cc = cc.to(device).float()                           
      batch_size = cc.shape[0]
      impu_noise = torch.empty(batch_size, p_f+1, device=device)
    
      cc_3, mask3 = next(iter(cc_data_loader3))
      cc_3 = cc_3[0:batch_size].to(device).float()
      mask3 = mask3[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data3 = imputer3(cc_3, mask3, impu_noise)
      cc_hat3 = cc_3.clone()
      cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3[:,np.logical_or(missing_col1,missing_col2)]

      update_critic = CriticUpdater_model(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
      update_critic(cc, cc_hat3)
      critic_updates += 1

      if critic_updates == n_critic:
        critic_updates = 0
        reconstruction_loss = 0
        
        impu_noise.normal_(mean=0,std=1)
        imputed_data3 = imputer3(cc_3, mask3, impu_noise)
        cc_hat3 = cc_3.clone()
        cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3[:,np.logical_or(missing_col1,missing_col2)]
        reconstruction_loss += l1_loss(imputed_data3, cc_3.float())
        loss = -critic(cc_hat3).mean() + weight*reconstruction_loss
        imputer3.zero_grad()
        loss.backward()
        imputer_optimizer.step()
    scheduler1.step()
    scheduler2.step()
  
  with torch.no_grad():
    imputer1.eval()
    imputer2.eval()
    imputer3.eval()
    inc_data_pat1 = torch.from_numpy(rawdata_inc1).to(device)
    inc_data_pat2 = torch.from_numpy(rawdata_inc2).to(device)
    inc_data_pat3 = torch.from_numpy(rawdata_inc3).to(device)
    mask1 = torch.from_numpy(missing_col1).to(device)
    mask2 = torch.from_numpy(missing_col2).to(device)
    mask3 = torch.from_numpy(np.logical_and(missing_col1,missing_col2)).to(device)
    for i in range(1):
      impu_noise = torch.empty(sum(np.logical_and(missing_row1,np.logical_not(missing_row2))), p_f+1, device=device)
      imputed_data = imputer1(inc_data_pat1.float(), mask1.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc1[:,missing_col1] = imputed_data.cpu().numpy()[:,missing_col1]
      rawdata[np.logical_and(missing_row1,np.logical_not(missing_row2))] = rawdata_inc1.copy()
      
      impu_noise = torch.empty(sum(np.logical_and(missing_row2,np.logical_not(missing_row1))), p_f+1, device=device)
      imputed_data = imputer2(inc_data_pat2.float(), mask2.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc2[:,missing_col2] = imputed_data.cpu().numpy()[:,missing_col2]
      rawdata[np.logical_and(missing_row2,np.logical_not(missing_row1))] = rawdata_inc2.copy()

      impu_noise = torch.empty(sum(np.logical_and(missing_row1,missing_row2)), p_f+1, device=device)
      imputed_data = imputer3(inc_data_pat3.float(), mask3.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data.cpu().numpy()[:,np.logical_or(missing_col1,missing_col2)]
      rawdata[np.logical_and(missing_row1, missing_row2)] = rawdata_inc3.copy()

  
  alist = []
  for j in range(1+MI):
    rawdata_pat1 = rawdata[np.logical_and(missing_row1,missing_row2==False),:].copy()
    train_pat1 = rawdata[np.logical_or(missing_row1==False,missing_row2),:].copy()
    train_pat1 = Generate_data_model_2(train_pat1, missing_col1)
    train_pat1_loader1 = DataLoader(train_pat1, batch_size=Batch_size, shuffle=True)
    train_pat1_loader2 = DataLoader(train_pat1, batch_size=Batch_size, shuffle=True)
    
    imputer1 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=p_f+1).to(device) 
    critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)    
    imputer_optimizer = optim.Adam(imputer1.parameters(), lr=ilr, betas=(.5, .9))
    critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
    scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
    scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
    
    critic_updates = 0
    for epoch in range(1,1+epochs):
      for cc, mask in train_pat1_loader1:      
        cc = cc.to(device).float()                           
        batch_size = cc.shape[0]
        impu_noise = torch.empty(batch_size, p_f+1, device=device)
      
        cc_1, mask1 = next(iter(train_pat1_loader2))
        cc_1 = cc_1[0:batch_size].to(device).float()
        mask1 = mask1[0:batch_size].to(device).float()
        impu_noise.normal_(mean=0,std=1)
        imputed_data1 = imputer1(cc_1, mask1, impu_noise)
        cc_hat1 = cc_1.clone()
        cc_hat1[:,missing_col1] = imputed_data1[:,missing_col1]
        update_critic = CriticUpdater_model(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
        update_critic(cc, cc_hat1)
        critic_updates += 1

        if critic_updates == n_critic:
          critic_updates = 0
          reconstruction_loss = 0

          impu_noise.normal_(mean=0,std=1)
          imputed_data1 = imputer1(cc_1, mask1, impu_noise)
          cc_hat1 = cc_1.clone()
          cc_hat1[:,missing_col1] = imputed_data1[:,missing_col1]
          reconstruction_loss += l1_loss(imputed_data1, cc_1.float())
          loss = -critic(cc_hat1).mean() + weight*reconstruction_loss
          imputer1.zero_grad()
          loss.backward()
          imputer_optimizer.step()
      scheduler1.step()
      scheduler2.step()
    
    with torch.no_grad():
      imputer1.eval()
      inc_data_pat1 = torch.from_numpy(rawdata_pat1).to(device)
      mask1 = torch.from_numpy(missing_col1).to(device)
      for i in range(1):
        torch.manual_seed(j)
        impu_noise = torch.empty(sum(np.logical_and(missing_row1,missing_row2==False)), p_f+1, device=device)
        imputed_data = imputer1(inc_data_pat1.float(), mask1.float(), impu_noise.normal_(mean=0,std=1)) 
        rawdata_pat1[:,missing_col1] = imputed_data.cpu().numpy()[:,missing_col1]
        rawdata[np.logical_and(missing_row1,missing_row2==False)] = rawdata_pat1.copy()

    rawdata_pat2 = rawdata[np.logical_and(missing_row1==False,missing_row2),:].copy()
    train_pat2 = rawdata[np.logical_or(missing_row1,missing_row2==False),:].copy()
    train_pat2 =  Generate_data_model_2(train_pat2, missing_col2)
    
    train_pat2_loader1 = DataLoader(train_pat2, batch_size=Batch_size, shuffle=True)
    train_pat2_loader2 = DataLoader(train_pat2, batch_size=Batch_size, shuffle=True)
    imputer2 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=p_f+1).to(device) 
    critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)
    imputer_optimizer = optim.Adam(imputer2.parameters(), lr=ilr, betas=(.5, .9))
    critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
    scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
    scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
    critic_updates = 0

    for epoch in range(1,1+epochs):
      for cc, mask in train_pat2_loader1:      
        cc = cc.to(device).float()                           
        batch_size = cc.shape[0]
        impu_noise = torch.empty(batch_size, p_f+1, device=device)
      
        cc_2, mask2 = next(iter(train_pat2_loader2))
        cc_2 = cc_2[0:batch_size].to(device).float()
        mask2 = mask2[0:batch_size].to(device).float()
        impu_noise.normal_(mean=0,std=1)
        imputed_data2 = imputer2(cc_2, mask2, impu_noise)
        cc_hat2 = cc_2.clone()
        cc_hat2[:,missing_col2] = imputed_data2[:,missing_col2]

        update_critic = CriticUpdater_model(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
        update_critic(cc, cc_hat2)
        critic_updates += 1

        if critic_updates == n_critic:
          critic_updates = 0
          reconstruction_loss = 0
          
          impu_noise.normal_(mean=0,std=1)
          imputed_data2 = imputer2(cc_2, mask2, impu_noise)
          cc_hat2 = cc_2.clone()
          cc_hat2[:,missing_col2] = imputed_data2[:,missing_col2]
          reconstruction_loss += l1_loss(imputed_data2, cc_2.float())
          loss = -critic(cc_hat2).mean() + weight*reconstruction_loss
          imputer2.zero_grad()
          loss.backward()
          imputer_optimizer.step()
      scheduler1.step()
      scheduler2.step()

    with torch.no_grad():
      imputer2.eval()
      inc_data_pat2 = torch.from_numpy(rawdata_pat2).to(device)
      mask2 = torch.from_numpy(missing_col2).to(device)
      for i in range(1):
        torch.manual_seed(j)
        impu_noise = torch.empty(sum(np.logical_and(missing_row1==False,missing_row2)), p_f+1, device=device)
        imputed_data = imputer2(inc_data_pat2.float(), mask2.float(), impu_noise.normal_(mean=0,std=1)) 
        rawdata_pat2[:,missing_col2] = imputed_data.cpu().numpy()[:,missing_col2]
        rawdata[np.logical_and(missing_row1==False,missing_row2)] = rawdata_pat2.copy()
    
    rawdata_pat3 = rawdata[np.logical_and(missing_row1,missing_row2),:].copy()
    train_pat3 = rawdata[np.logical_or(missing_row1==False,missing_row2==False),:].copy()
    train_pat3 =  Generate_data_model_2(train_pat3, np.logical_or(missing_col1,missing_col2))
    
    train_pat3_loader1 = DataLoader(train_pat3, batch_size=Batch_size, shuffle=True)
    train_pat3_loader2 = DataLoader(train_pat3, batch_size=Batch_size, shuffle=True)
    imputer3 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=p_f+1).to(device) 
    critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)
    imputer_optimizer = optim.Adam(imputer3.parameters(), lr=ilr, betas=(.5, .9))
    critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
    scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
    scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
    critic_updates = 0

    for epoch in range(1,1+epochs):
      for cc, mask in train_pat3_loader1:      
        cc = cc.to(device).float()                           
        batch_size = cc.shape[0]
        impu_noise = torch.empty(batch_size, p_f+1, device=device)
      
        cc_3, mask3 = next(iter(train_pat3_loader2))
        cc_3 = cc_3[0:batch_size].to(device).float()
        mask3 = mask3[0:batch_size].to(device).float()
        impu_noise.normal_(mean=0,std=1)
        imputed_data3 = imputer3(cc_3, mask3, impu_noise)
        cc_hat3 = cc_3.clone()
        cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3[:,np.logical_or(missing_col1,missing_col2)]

        update_critic = CriticUpdater_model(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
        update_critic(cc, cc_hat3)
        critic_updates += 1

        if critic_updates == n_critic:
          critic_updates = 0
          reconstruction_loss = 0
          
          impu_noise.normal_(mean=0,std=1)
          imputed_data3 = imputer3(cc_3, mask3, impu_noise)
          cc_hat3 = cc_3.clone()
          cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3[:,np.logical_or(missing_col1,missing_col2)]
          reconstruction_loss += l1_loss(imputed_data3, cc_3.float())
          loss = -critic(cc_hat3).mean() + weight*reconstruction_loss
          imputer3.zero_grad()
          loss.backward()
          imputer_optimizer.step()
      scheduler1.step()
      scheduler2.step()

    with torch.no_grad():
      imputer3.eval()
      inc_data_pat3 = torch.from_numpy(rawdata_pat3).to(device)
      mask3 = torch.from_numpy(np.logical_or(missing_col1,missing_col2)).to(device)
      for i in range(1):
        torch.manual_seed(j)
        impu_noise = torch.empty(sum(np.logical_and(missing_row1,missing_row2)), p_f+1, device=device)
        imputed_data = imputer3(inc_data_pat3.float(), mask3.float(), impu_noise.normal_(mean=0,std=1)) 
        rawdata_pat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data.cpu().numpy()[:,np.logical_or(missing_col1,missing_col2)]
        rawdata[np.logical_and(missing_row1,missing_row2)] = rawdata_pat3.copy()

    if j > 0:
      alist.append(rawdata.copy())
  return alist

## iterative approach, row pattern, 2, joint training

In [ ]:
class CriticUpdater_model_3:
  def __init__(self, critic1, critic2, critic3, critic_optimizer, batch_size1, batch_size2, batch_size3, gp_lambda=10, device='cuda:0'):
    self.critic1 = critic1
    self.critic2 = critic2
    self.critic3 = critic3
    self.critic_optimizer = critic_optimizer
    self.gp_lambda = gp_lambda
    # Interpolation coefficient
    self.eps1 = torch.empty(batch_size1, 1,device=device)
    self.eps2 = torch.empty(batch_size2, 1,device=device)
    self.eps3 = torch.empty(batch_size3, 1,device=device)
    # For computing the gradient penalty
    self.ones1 = torch.ones(batch_size1, 1).to(device)
    self.ones2 = torch.ones(batch_size2, 1).to(device)
    self.ones3 = torch.ones(batch_size3, 1).to(device)

  def __call__(self, real1, real2, real3, fake1, fake2, fake3, device='cuda:0'):
    real1 = real1.detach()
    real2 = real2.detach()
    real3 = real3.detach()
    fake1 = fake1.detach()
    fake2 = fake2.detach()
    fake3 = fake3.detach()
    self.critic1.zero_grad()
    self.critic2.zero_grad()
    self.critic3.zero_grad()
    self.eps1.uniform_(0, 1)
    interp1 = (self.eps1 * real1 + (1 - self.eps1) * fake1).to(device).requires_grad_()
    self.eps2.uniform_(0, 1)
    interp2 = (self.eps2 * real2 + (1 - self.eps2) * fake2).to(device).requires_grad_()
    self.eps3.uniform_(0, 1)
    interp3 = (self.eps3 * real3 + (1 - self.eps3) * fake3).to(device).requires_grad_()
    grad_d1 = grad(outputs=self.critic1(interp1.float()), inputs=interp1, grad_outputs=self.ones1,create_graph=True)[0].view(real1.shape[0], -1)  #grad_outputs=self.ones,create_graph=True
    grad_d2 = grad(outputs=self.critic2(interp2.float()), inputs=interp2, grad_outputs=self.ones2,create_graph=True)[0].view(real2.shape[0], -1)
    grad_d3 = grad(outputs=self.critic3(interp3.float()), inputs=interp3, grad_outputs=self.ones3,create_graph=True)[0].view(real3.shape[0], -1)
    grad_penalty1 = ((grad_d1.norm(dim=1) - 1)**2).mean() * self.gp_lambda
    grad_penalty2 = ((grad_d2.norm(dim=1) - 1)**2).mean() * self.gp_lambda
    grad_penalty3 = ((grad_d3.norm(dim=1) - 1)**2).mean() * self.gp_lambda
    w_dist1 = self.critic1(fake1.float()).mean() - self.critic1(real1.float()).mean()
    w_dist2 = self.critic2(fake2.float()).mean() - self.critic2(real2.float()).mean()
    w_dist3 = self.critic3(fake3.float()).mean() - self.critic3(real3.float()).mean()
    loss = w_dist1 + w_dist2 + w_dist3 + grad_penalty1 + grad_penalty2 + grad_penalty3 
    loss.backward()
    self.critic_optimizer.step()

In [ ]:
def mlp_MIGAN1_iterative3(missing_data, missing_row1,missing_row2,missing_col1,missing_col2, n=200, p_f=250, device='cuda:0', Batch_size=16, ilr=1e-3, clr=1e-3, gamma=0.2, weight=0.1, gp_lambda=10, epochs=300, MI=10):
  rawdata = missing_data.copy()
  rawdata_inc1 = rawdata[np.logical_and(missing_row1,np.logical_not(missing_row2)), :].copy()
  rawdata_inc2 = rawdata[np.logical_and(missing_row2,np.logical_not(missing_row1)), :].copy()
  rawdata_inc3 = rawdata[np.logical_and(missing_row1,missing_row2), :].copy()
  cc = torch.from_numpy(rawdata.copy()[np.logical_or(missing_row1,missing_row2)==0, :])

  cc_data = Generate_data_model(cc)
  cc_data_pat1 =  Generate_data_model_2(cc, missing_col1)
  cc_data_pat2 =  Generate_data_model_2(cc, missing_col2)
  cc_data_pat3 =  Generate_data_model_2(cc, np.logical_or(missing_col1,missing_col2))

  cc_data_loader = DataLoader(cc_data, batch_size=Batch_size, shuffle=True)
  cc_data_loader1 = DataLoader(cc_data_pat1, batch_size=Batch_size, shuffle=True)
  cc_data_loader2 = DataLoader(cc_data_pat2, batch_size=Batch_size, shuffle=True)
  cc_data_loader3 = DataLoader(cc_data_pat3, batch_size=Batch_size, shuffle=True)
  
  imputer1 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=p_f+1).to(device)
  imputer2 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=p_f+1).to(device)
  imputer3 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=p_f+1).to(device)  

  critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)    
  imputer_params = list(imputer1.parameters()) + list(imputer2.parameters()) + list(imputer3.parameters())
  imputer_optimizer = optim.Adam(imputer_params, lr=ilr, betas=(.5, .9))
  critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
  scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
  scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
  mse_loss = torch.nn.MSELoss(reduction='mean')
  l1_loss = nn.L1Loss(reduction='mean')

  critic_updates = 0
  n_critic = 5

  for epoch in range(1,1+epochs):
    for cc in cc_data_loader:      
      cc = cc.to(device).float()                           
      batch_size = cc.shape[0]
      impu_noise = torch.empty(batch_size, p_f+1, device=device)
     
      cc_1, mask1 = next(iter(cc_data_loader1))
      cc_1 = cc_1[0:batch_size].to(device).float()
      mask1 = mask1[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data1 = imputer1(cc_1, mask1, impu_noise)
      cc_hat1 = cc_1.clone()
      cc_hat1[:,missing_col1] = imputed_data1[:,missing_col1]

      cc_2, mask2 = next(iter(cc_data_loader2))
      cc_2 = cc_2[0:batch_size].to(device).float()
      mask2 = mask2[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data2 = imputer2(cc_2, mask2, impu_noise)
      cc_hat2 = cc_2.clone()
      cc_hat2[:,missing_col2] = imputed_data2[:,missing_col2] 

      cc_3, mask3 = next(iter(cc_data_loader3))
      cc_3 = cc_3[0:batch_size].to(device).float()
      mask3 = mask3[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data3 = imputer3(cc_3, mask3, impu_noise)
      cc_hat3 = cc_3.clone()
      cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3[:,np.logical_or(missing_col1,missing_col2)]
      
      update_critic = CriticUpdater_model_2(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
      update_critic(cc, cc_hat1, cc_hat2, cc_hat3)
      critic_updates += 1

      if critic_updates == n_critic:
        critic_updates = 0
        reconstruction_loss = 0

        impu_noise.normal_(mean=0,std=1)
        imputed_data1 = imputer1(cc_1, mask1, impu_noise)
        cc_hat1 = cc_1.clone()
        cc_hat1[:,missing_col1] = imputed_data1[:,missing_col1]
        reconstruction_loss += l1_loss(imputed_data1, cc_1.float())
        
        impu_noise.normal_(mean=0,std=1)
        imputed_data2 = imputer2(cc_2, mask2, impu_noise)
        cc_hat2 = cc_2.clone()
        cc_hat2[:,missing_col2] = imputed_data2[:,missing_col2]
        reconstruction_loss += l1_loss(imputed_data2, cc_2.float())

        impu_noise.normal_(mean=0,std=1)
        imputed_data3 = imputer3(cc_3, mask3, impu_noise)
        cc_hat3 = cc_3.clone()
        cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3[:,np.logical_or(missing_col1,missing_col2)]
        reconstruction_loss += l1_loss(imputed_data3, cc_3.float())

        loss = -critic(cc_hat1).mean()-critic(cc_hat2).mean()-critic(cc_hat3).mean() + weight*reconstruction_loss
        
        imputer1.zero_grad()
        imputer2.zero_grad()
        imputer3.zero_grad()
        loss.backward()
        imputer_optimizer.step()

    scheduler1.step()
    scheduler2.step()

  with torch.no_grad():
    imputer1.eval()
    imputer2.eval()
    imputer3.eval()
    inc_data_pat1 = torch.from_numpy(rawdata_inc1).to(device)
    inc_data_pat2 = torch.from_numpy(rawdata_inc2).to(device)
    inc_data_pat3 = torch.from_numpy(rawdata_inc3).to(device)
    mask1 = torch.from_numpy(missing_col1).to(device)
    mask2 = torch.from_numpy(missing_col2).to(device)
    mask3 = torch.from_numpy(np.logical_and(missing_col1,missing_col2)).to(device)
    for i in range(1):
      impu_noise = torch.empty(sum(np.logical_and(missing_row1,np.logical_not(missing_row2))), p_f+1, device=device)
      imputed_data = imputer1(inc_data_pat1.float(), mask1.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc1[:,missing_col1] = imputed_data.cpu().numpy()[:,missing_col1]
      rawdata[np.logical_and(missing_row1,np.logical_not(missing_row2))] = rawdata_inc1.copy()
      
      impu_noise = torch.empty(sum(np.logical_and(missing_row2,np.logical_not(missing_row1))), p_f+1, device=device)
      imputed_data = imputer2(inc_data_pat2.float(), mask2.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc2[:,missing_col2] = imputed_data.cpu().numpy()[:,missing_col2]
      rawdata[np.logical_and(missing_row2,np.logical_not(missing_row1))] = rawdata_inc2.copy()

      impu_noise = torch.empty(sum(np.logical_and(missing_row1,missing_row2)), p_f+1, device=device)
      imputed_data = imputer3(inc_data_pat3.float(), mask3.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data.cpu().numpy()[:,np.logical_or(missing_col1,missing_col2)]
      rawdata[np.logical_and(missing_row1, missing_row2)] = rawdata_inc3.copy()
  
  alist = []
  for j in range(2+MI):
    rawdata_pat1 = rawdata[np.logical_and(missing_row1,missing_row2==False),:].copy()
    rawdata_pat2 = rawdata[np.logical_and(missing_row1==False,missing_row2),:].copy()
    rawdata_pat3 = rawdata[np.logical_and(missing_row1,missing_row2),:].copy()
    train_pat1 = rawdata[np.logical_or(missing_row1==False,missing_row2),:].copy()
    train_pat1 = Generate_data_model_2(train_pat1, missing_col1)
    train_pat2 = rawdata[np.logical_or(missing_row1,missing_row2==False),:].copy()
    train_pat2 =  Generate_data_model_2(train_pat2, missing_col2)
    train_pat3 = rawdata[np.logical_or(missing_row1==False,missing_row2==False),:].copy()
    train_pat3 =  Generate_data_model_2(train_pat3, np.logical_or(missing_col1,missing_col2))

    train_pat1_loader1 = DataLoader(train_pat1, batch_size=Batch_size, shuffle=True)
    train_pat1_loader2 = DataLoader(train_pat1, batch_size=Batch_size, shuffle=True)
    train_pat2_loader1 = DataLoader(train_pat2, batch_size=Batch_size, shuffle=True)
    train_pat2_loader2 = DataLoader(train_pat2, batch_size=Batch_size, shuffle=True)
    train_pat3_loader1 = DataLoader(train_pat3, batch_size=Batch_size, shuffle=True)
    train_pat3_loader2 = DataLoader(train_pat3, batch_size=Batch_size, shuffle=True)

    imputer1 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=p_f+1).to(device) 
    critic1 = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)  
    imputer2 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=p_f+1).to(device) 
    critic2 = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)
    imputer3 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=p_f+1).to(device) 
    critic3 = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)
    
    imputer_params = list(imputer1.parameters()) + list(imputer2.parameters()) + list(imputer3.parameters())
    critic_params = list(critic1.parameters()) + list(critic2.parameters()) + list(critic3.parameters())
    imputer_optimizer = optim.Adam(imputer_params, lr=ilr, betas=(.5, .9))
    critic_optimizer = optim.Adam(critic_params, lr=clr, betas=(.5, .9))
    scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
    scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)

    critic_updates = 0
    for epoch in range(1,1+epochs):
      for cc_1, mask1 in train_pat1_loader1:      
        cc_1 = cc_1.to(device).float()                           
        batch_size1 = cc_1.shape[0]
        impu_noise1 = torch.empty(batch_size1, p_f+1, device=device)
      
        cc_hat1, mask1 = next(iter(train_pat1_loader2))
        cc_hat1 = cc_hat1[0:batch_size1].to(device).float()
        mask1 = mask1[0:batch_size1].to(device).float()
        imputed_data1 = imputer1(cc_hat1, mask1, impu_noise1.normal_(mean=0,std=1))
        cc_hat1[:,missing_col1] = imputed_data1[:,missing_col1]
        
        cc_2, mask2 = next(iter(train_pat2_loader1))
        batch_size2 = cc_2.shape[0]
        impu_noise2 = torch.empty(batch_size2, p_f+1, device=device)
        cc_2 = cc_2[0:batch_size2].to(device).float()
        cc_hat2, mask2 = next(iter(train_pat2_loader2))
        cc_hat2 = cc_hat2[0:batch_size2].to(device).float()
        mask2 = mask2[0:batch_size2].to(device).float()
        imputed_data2 = imputer2(cc_hat2, mask2, impu_noise2.normal_(mean=0,std=1))
        cc_hat2[:,missing_col2] = imputed_data2[:,missing_col2]
        
        cc_3, mask3 = next(iter(train_pat3_loader1))
        batch_size3 = cc_3.shape[0]
        impu_noise3 = torch.empty(batch_size3, p_f+1, device=device)
        cc_3 = cc_3[0:batch_size3].to(device).float()
        cc_hat3, mask3 = next(iter(train_pat3_loader2))
        cc_hat3 = cc_hat3[0:batch_size3].to(device).float()
        mask3 = mask3[0:batch_size3].to(device).float()
        imputed_data3 = imputer3(cc_hat3, mask3, impu_noise3.normal_(mean=0,std=1))
        cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3[:,np.logical_or(missing_col1,missing_col2)]

        update_critic = CriticUpdater_model_3(critic1, critic2, critic3, critic_optimizer, batch_size1, batch_size2, batch_size3, device=device, gp_lambda=gp_lambda)
        update_critic(cc_1, cc_2, cc_3, cc_hat1, cc_hat2, cc_hat3)
        critic_updates += 1

        if critic_updates == n_critic:
          critic_updates = 0
          reconstruction_loss = 0

          imputed_data1 = imputer1(cc_hat1, mask1, impu_noise1.normal_(mean=0,std=1))
          cc_hat1[:,missing_col1] = imputed_data1[:,missing_col1]
          reconstruction_loss += l1_loss(imputed_data1, cc_1.float())
          
          imputed_data2 = imputer2(cc_hat2, mask2, impu_noise2.normal_(mean=0,std=1))
          cc_hat2[:,missing_col2] = imputed_data2[:,missing_col2]
          reconstruction_loss += l1_loss(imputed_data2, cc_2.float())
          
          imputed_data3 = imputer3(cc_hat3, mask3, impu_noise3.normal_(mean=0,std=1))
          cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3[:,np.logical_or(missing_col1,missing_col2)]
          reconstruction_loss += l1_loss(imputed_data3, cc_3.float())

          loss = -critic1(cc_hat1).mean()-critic2(cc_hat2).mean()-critic3(cc_hat3).mean() + weight*reconstruction_loss
          imputer_optimizer.zero_grad()
          loss.backward()
          imputer_optimizer.step()
      scheduler1.step()
      scheduler2.step()
    
    with torch.no_grad():
      imputer1.eval()
      imputer2.eval()
      imputer3.eval()
      inc_data_pat1 = torch.from_numpy(rawdata_pat1).to(device)
      inc_data_pat2 = torch.from_numpy(rawdata_pat2).to(device)
      inc_data_pat3 = torch.from_numpy(rawdata_pat3).to(device)
      mask1 = torch.from_numpy(missing_col1).to(device)
      mask2 = torch.from_numpy(missing_col2).to(device)
      mask3 = torch.from_numpy(np.logical_or(missing_col1,missing_col2)).to(device)
      for i in range(1):
        torch.manual_seed(j)
        impu_noise = torch.empty(sum(np.logical_and(missing_row1,missing_row2==False)), p_f+1, device=device)
        imputed_data = imputer1(inc_data_pat1.float(), mask1.float(), impu_noise.normal_(mean=0,std=1)) 
        rawdata_pat1[:,missing_col1] = imputed_data.cpu().numpy()[:,missing_col1]
        rawdata[np.logical_and(missing_row1,missing_row2==False)] = rawdata_pat1.copy()
        
        impu_noise = torch.empty(sum(np.logical_and(missing_row1==False,missing_row2)), p_f+1, device=device)
        imputed_data = imputer2(inc_data_pat2.float(), mask2.float(), impu_noise.normal_(mean=0,std=1)) 
        rawdata_pat2[:,missing_col2] = imputed_data.cpu().numpy()[:,missing_col2]
        rawdata[np.logical_and(missing_row1==False,missing_row2)] = rawdata_pat2.copy()
        
        impu_noise = torch.empty(sum(np.logical_and(missing_row1,missing_row2)), p_f+1, device=device)
        imputed_data = imputer3(inc_data_pat3.float(), mask3.float(), impu_noise.normal_(mean=0,std=1)) 
        rawdata_pat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data.cpu().numpy()[:,np.logical_or(missing_col1,missing_col2)]
        rawdata[np.logical_and(missing_row1,missing_row2)] = rawdata_pat3.copy()

    if j > 1:
      alist.append(rawdata.copy())
  return alist

## test

In [ ]:
Model_2_SD=[]
Model_2_SE_list=[]
Model_2_SE=0
Model_2_MSE=0
Model_2_bias=0
Model_2_coverage=0
Model_2_time=0

a0=1
a1=-2
a2=3
a3=0
a4=2
a5=-2
rho=0.95
sigma1=0.1
sigma2=0.5
Monte_Carlo =100
MI=10
batch_size=256
ilr=1e-3 
clr=1e-3 
gamma=0.9
weight=0.1 
gp_lambda=10 
epochs=300

for iteration in range(1,1+Monte_Carlo):
  obs,mis1,mis2,missing_row1,missing_row2,missing_col1,missing_col2,y=data_G_linear(n=n,p=p,true_beta=true_beta,seed=iteration,a0=a0,a1=a1,a2=a2,a3=a3,a4=a4,a5=a5,rho=rho,sigma1=sigma1,sigma2=sigma2)
  truth1=mis1[missing_row1==True,:].copy()
  truth2=mis2[missing_row2==True,:].copy()

  mis1[missing_row1==True,:]=0
  mis2[missing_row2==True,:]=0
  rawdata = np.concatenate((obs,mis1,mis2,y),axis=1).copy()
  
  s=time.time()
  alist=[]
  for i in range(MI):
    data = mlp_MIGAN1(rawdata,missing_row1,missing_row2,missing_col1,missing_col2,n=n,p_f=p,Batch_size=batch_size,ilr=ilr,clr=clr,gamma=gamma,weight=weight,gp_lambda=gp_lambda,epochs=epochs,MI_seed=i)[0]
    alist.append(data)

  # alist = mlp_MIGAN1_iterative2(rawdata,missing_row1,missing_row2,missing_col1,missing_col2,n=n,p_f=p,Batch_size=batch_size,ilr=ilr,clr=clr,gamma=gamma,weight=weight,gp_lambda=gp_lambda,epochs=epochs,MI=MI)

  mse,b,c,se=data_to_stats(alist,true_beta,truth1,truth2,missing_row1,missing_row2,missing_col1,missing_col2)
  bias,cover=Performance(b,c,true_beta,alpha)
  print('Model_2 cost ',time.time()-s,' sec \n','MSE:', mse,'; Bias: ',bias,'; Coverage: ',cover)
  Model_2_SE_list.append(se)
  Model_2_SE=(Model_2_SE*(iteration-1)+se)/iteration
  Model_2_SD.append(b[0])
  Model_2_MSE=(Model_2_MSE*(iteration-1)+mse)/iteration
  Model_2_bias=(Model_2_bias*(iteration-1)+bias)/iteration
  Model_2_coverage=(Model_2_coverage*(iteration-1)+cover)/iteration
  Model_2_time = (Model_2_time*(iteration-1)+time.time()-s)/iteration

  if iteration%5==0:
    print('iteration: ',iteration, 'Model_2_MSE: ',Model_2_MSE,'Model_2_bias: ',Model_2_bias,'Model_2_coverage: ',Model_2_coverage,'Model_2_SE: ',Model_2_SE)
    
    Model_2_StD=np.std(Model_2_SD, dtype=np.float64)
    print('Model_2_StD: ',Model_2_StD)
    Model_2_SE_median=statistics.median(Model_2_SE_list)
    print('Model_2 average cost:', Model_2_time/MI)

print('n:', n, ' p: ',p, ' Monte Carlo:',Monte_Carlo, 'MI:',MI,'batch_size:',batch_size,'ilr:',ilr,'clr:',clr,'gamma:',gamma,'weight:',weight,'gp_lambda:',gp_lambda,'epochs:',epochs)
print('a0=',a0,' a1=', a1,' a2=',a2,' a3=',a3,' a4=',a4,' a5=',a5,' rho=',rho,' sigma1=',sigma1,' sigma2=',sigma2)